In [1]:
!lscpu

Architecture:                x86_64
  CPU op-mode(s):            32-bit, 64-bit
  Address sizes:             48 bits physical, 48 bits virtual
  Byte Order:                Little Endian
CPU(s):                      8
  On-line CPU(s) list:       0-7
Vendor ID:                   AuthenticAMD
  Model name:                AMD EPYC 7B12
    CPU family:              23
    Model:                   49
    Thread(s) per core:      2
    Core(s) per socket:      4
    Socket(s):               1
    Stepping:                0
    BogoMIPS:                4499.99
    Flags:                   fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pg
                             e mca cmov pat pse36 clflush mmx fxsr sse sse2 ht s
                             yscall nx mmxext fxsr_opt pdpe1gb rdtscp lm constan
                             t_tsc rep_good nopl nonstop_tsc cpuid extd_apicid t
                             sc_known_freq pni pclmulqdq ssse3 fma cx16 sse4_1 s
                             se4_2 

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import time
from collections import deque
from tqdm import tqdm
from typing import Dict, List, Tuple
import math

In [4]:
import random
import numpy as np
from numpy import ndarray

In [5]:
# Set seeds for reproducibility
# --- Fixed parameters ---
np.random.seed(42)  # set seed for reproducibility

In [6]:
!echo "Rows: $(printf "%'d" $(wc -l < /content/drive/MyDrive/datasets/dataSearch/sift_base_augmented.txt))"
!echo "Columns: $(head -n 1 /content/drive/MyDrive/datasets/dataSearch/sift_base_augmented.txt | awk '{print NF}')"

Rows: 6,000,000
Columns: 20


In [7]:
# Define the dataset path
dataset_path = '/content/drive/MyDrive/datasets/dataSearch/sift_base_augmented.txt'

# Load the dataset
# If the file has a different delimiter (e.g., commas), can set delimiter=',' accordingly.
dataset = np.loadtxt(dataset_path)
total_points = len(dataset)

In [8]:
# Data sizes:
EB = 0
SB = 10_000    # Small Batch: 10K points
MB = 100_000   # Medium Batch: 100K points
LB = 1_000_000 # Large Batch: 1M points

# Query sizes:
EQ = 0
SQ = 20        # Small Query: 20 queries
MQ = 100       # Medium Query: 100 queries
LQ = 500       # Large Query: 500 queries

# === Define the test combinations ===
# Each tuple is of the form (data_size, query_size)
test_combinations = [
    (SB, EQ), (MB, MQ), (EB, MQ), (MB, EQ), (SB, MQ), (MB, MQ), (LB, SQ), (MB, LQ), (MB, EQ), (EB, SQ)
 ]

In [35]:
# Compute total number of required points across all test combinations
required_points = sum(data_size for data_size, _ in test_combinations)

# Sample from the full dataset without replacement
if required_points > total_points:
    raise ValueError(f"Need {required_points} points but only {total_points} available.")

sampled_indices = np.random.choice(total_points, size=required_points, replace=False)
combined_dataset = dataset[sampled_indices]


In [36]:
# Determine query source based on the first combination's batch size
use_full_dataset_for_queries = test_combinations[0][0] == 0

query_source = dataset if use_full_dataset_for_queries else combined_dataset

# Now generate queries
queries_list = []
st_idx = 0  # Start index for data slicing

for data_size, query_size in test_combinations:
    end_idx = st_idx + data_size
    current_data = combined_dataset[st_idx:end_idx]
    st_idx = end_idx

    if query_size <= len(query_source):
        query_indices = np.random.choice(len(query_source), size=query_size, replace=False)
        queries = query_source[query_indices]
    else:
        raise ValueError(
            f"Query size {query_size} exceeds available points "
            f"in query source (len={len(query_source)})"
        )

    queries_list.append(queries)

In [11]:
print(len(queries_list))
print(len(combined_dataset))

10
1520000


----------------------------------------------------------------------

### VP_MWV
---

In [ ]:
import numpy as np

class VPNode:
    def __init__(self, vp=None):
        # Vantage point
        self.vp = vp

        # Distance bounds
        self.low_near = float('inf')
        self.low_far = 0
        self.high_near = float('inf')
        self.high_far = 0

        # Child nodes
        self.lowChild = None
        self.highChild = None

        # Bucket for leaf nodes
        self.bucket = []

    def is_leaf(self):
        return self.lowChild is None and self.highChild is None

class VPTree:
    def __init__(self, bucket_size=50, distance_fn=None):
        """
        Initialize a VP-Tree

        Parameters:
        - bucket_size: Square root of maximum bucket size (b where b² is max bucket size)
        - distance_fn: Distance function to use (defaults to Euclidean)
        """
        self.bucket_size = bucket_size
        self.root = None

        # Default to Euclidean distance if none provided
        self.distance_fn = distance_fn if distance_fn is not None else self._euclidean_distance

    def _euclidean_distance(self, x, y):
        """Calculate Euclidean distance between two points"""
        x = np.asarray(x)
        y = np.asarray(y)
        diff = x - y
        return np.sqrt(np.dot(diff, diff))

    def insert(self, x):
        """Insert a new datapoint x into the VP-tree"""
        # If tree is empty, create root with this point as VP
        if self.root is None:
            self.root = VPNode(vp=x)
            return

        p = self.root

        # Traverse until we reach a leaf node
        while not p.is_leaf():
            # Calculate distance from x to current vantage point
            dist = self.distance_fn(x, p.vp)

            # Determine which child to go to
            if dist < (p.low_far + p.high_near) / 2:
                # Update distance bounds
                p.low_far = max(dist, p.low_far)
                p.low_near = min(dist, p.low_near)

                # Go to low (near) child
                p = p.lowChild
            else:
                # Update distance bounds
                p.high_far = max(dist, p.high_far)
                p.high_near = min(dist, p.high_near)

                # Go to high (far) child
                p = p.highChild

        # Add x to bucket leaf node p
        p.bucket.append(x)

        # If bucket exceeds b², split it
        if len(p.bucket) > self.bucket_size**2:
            self._split_node(p)

    def _select_vantage_point(self, points):
        """Select a vantage point from the bucket
        Can use various methods; simple random selection is shown here.
        """
        # For simplicity, using the first point as VP
        # Could implement more sophisticated selection methods
        if points:
            return points[0]
        return None

    def _split_node(self, p):
        """Split a node using the CART-inspired weighted variance method"""
        points = p.bucket.copy()

        # Select vantage point from bucket
        vp = self._select_vantage_point(points)
        points.remove(vp)  # Remove VP from points list

        # Calculate distances to vantage point
        distances = [(self.distance_fn(vp, point), point) for point in points]

        # Sort by distance
        distances.sort(key=lambda x: x[0])
        sorted_points = [point for _, point in distances]
        sorted_dists = [dist for dist, _ in distances]

        # Find optimal split using weighted variance minimization
        n = len(sorted_points)
        best_split = 0
        min_weighted_variance = float('inf')

        for s in range(1, n):
            # Calculate variances for 1:s and s:n
            if s > 0:
                var_left = np.var(sorted_dists[:s]) if s > 1 else 0
            else:
                var_left = 0

            if n-s > 0:
                var_right = np.var(sorted_dists[s:]) if n-s > 1 else 0
            else:
                var_right = 0

            # Calculate weighted variance
            weighted_var = s * var_left + (n-s) * var_right

            if weighted_var < min_weighted_variance:
                min_weighted_variance = weighted_var
                best_split = s

        # Create new children nodes
        p.lowChild = VPNode(vp=vp)
        p.highChild = VPNode(vp=self._select_vantage_point(sorted_points[best_split:]))

        # Distribute points to children based on best split
        p.lowChild.bucket = sorted_points[:best_split]
        p.highChild.bucket = sorted_points[best_split:]

        # Initialize distance bounds
        if p.lowChild.bucket:
            low_dists = sorted_dists[:best_split]
            p.low_near = min(low_dists)
            p.low_far = max(low_dists)

        if p.highChild.bucket:
            # If we have a vantage point for the high child, calculate distances to it
            high_dists = [self.distance_fn(p.highChild.vp, point) for point in p.highChild.bucket]
            p.high_near = min(high_dists) if high_dists else float('inf')
            p.high_far = max(high_dists) if high_dists else 0

        # Clear the bucket as it's no longer a leaf
        p.bucket = []

    def nearest_neighbor(self, query):
        """Optimized nearest neighbor search with batch processing for bucket points"""
        best_dist = float('inf')
        best_point = None

        # Use a stack instead of recursion to reduce overhead
        # Each entry is (node, min_possible_dist)
        stack = [(self.root, 0)]

        while stack:
            node, min_dist = stack.pop()

            # Skip this node if it cannot contain a better solution
            if min_dist >= best_dist:
                continue

            # Check vantage point distance
            if node.vp is not None:
                # Calculate distance to vantage point
                diff = np.subtract(query, node.vp)
                dist_to_vp = np.sqrt(np.dot(diff, diff))

                if dist_to_vp < best_dist:
                    best_dist = dist_to_vp
                    best_point = node.vp

            # If leaf node, check all points in bucket using batch processing
            if node.is_leaf():
                if node.bucket:
                    # Convert bucket to numpy array once
                    bucket_array = np.array(node.bucket)

                    # Vectorized distance calculation
                    query_array = np.array(query)
                    diffs = bucket_array - query_array
                    squared_dists = np.sum(diffs**2, axis=1)

                    # Find minimum distance point without unnecessary sqrt operations
                    min_idx = np.argmin(squared_dists)
                    min_squared_dist = squared_dists[min_idx]

                    # Only compute sqrt for potentially better distance
                    if min_squared_dist < best_dist**2:
                        min_dist = np.sqrt(min_squared_dist)
                        if min_dist < best_dist:
                            best_dist = min_dist
                            best_point = node.bucket[min_idx]
                continue

            # Calculate threshold
            threshold = (node.low_far + node.high_near) / 2

            # Calculate bounds for both children
            if dist_to_vp < threshold:
                # Query is in near region
                # Add far child to stack first (so near child gets processed first)
                if node.highChild:
                    far_bound = max(0, dist_to_vp - node.low_far)
                    if far_bound < best_dist:
                        stack.append((node.highChild, far_bound))

                # Add near child with zero bound (must contain query)
                if node.lowChild:
                    stack.append((node.lowChild, 0))
            else:
                # Query is in far region
                # Add near child to stack first (so far child gets processed first)
                if node.lowChild:
                    near_bound = max(0, node.high_near - dist_to_vp)
                    if near_bound < best_dist:
                        stack.append((node.lowChild, near_bound))

                # Add far child with zero bound (must contain query)
                if node.highChild:
                    stack.append((node.highChild, 0))

        return best_dist, best_point

------------------------------------------------

**------------VP_MWV (Run 1)----------------**

In [ ]:
results = []
st_idx = 0  # reset the dataset slicing index for processing test cases

vptree = VPTree()

for idx, (data_size, query_size) in enumerate(test_combinations):
    print(f"\n{idx+1}: === Testing with Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # Slice the current data chunk for this test.
    end_idx = st_idx + data_size
    current_data = combined_dataset[st_idx:end_idx]
    st_idx = end_idx

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[idx]

    # --- Build the VP tree ---
    build_start = time.perf_counter()
    for pt in tqdm(current_data):
        vptree.insert(pt)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc="Querying"):
        search_start = time.perf_counter()
        r_min, nn_pt = vptree.nearest_neighbor(query=query)
        search_time += time.perf_counter() - search_start

    results.append(((data_size, query_size), build_time, search_time))
    print(f"Build Time: {build_time:.2f} sec, Search Time: {search_time:.2f} sec")

# ----------------------------
# Summary of Results
# ----------------------------
print("\n=== Summary of Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    print(f"Data: {data_size:>7} points, Queries: {query_size:>4} => Build: {b_time:.2f} sec, Search: {s_time:.2f} sec")

print("--------------------------------------------")
# ----------------------------
# Total Results
# ----------------------------
build_time_total = search_time_total = 0.0
print("\n=== Summary of VP MWV Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    build_time_total += b_time
    search_time_total += s_time
print(f"\nTotal Build Time: {build_time_total:.2f} sec, Total Search Time: {search_time_total:.2f} sec")

print("--------------------------------------------")
total_time = build_time_total + search_time_total
print(f"Total Time: {total_time:.2f}")


1: === Testing with Data Size: 10,000 points, Query Size: 0 queries ===


100%|██████████| 10000/10000 [00:01<00:00, 6496.07it/s]
Querying: 0it [00:00, ?it/s]


Build Time: 1.54 sec, Search Time: 0.00 sec

2: === Testing with Data Size: 100,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:05<00:00, 16.80it/s]


Build Time: 26.79 sec, Search Time: 5.91 sec

3: === Testing with Data Size: 0 points, Query Size: 100 queries ===


...

Build Time: 36.43 sec, Search Time: 48.58 sec

9: === Testing with Data Size: 100,000 points, Query Size: 0 queries ===


100%|██████████| 100000/100000 [00:40<00:00, 2456.00it/s]
Querying: 0it [00:00, ?it/s]


Build Time: 40.72 sec, Search Time: 0.00 sec

10: === Testing with Data Size: 0 points, Query Size: 20 queries ===


0it [00:00, ?it/s]
Querying: 100%|██████████| 20/20 [00:00<00:00, 69.95it/s]

Build Time: 0.00 sec, Search Time: 0.28 sec

=== Summary of Tree Tests ===
Data:   10000 points, Queries:    0 => Build: 1.54 sec, Search: 0.00 sec
Data:  100000 points, Queries:  100 => Build: 26.79 sec, Search: 5.91 sec
Data:       0 points, Queries:  100 => Build: 0.00 sec, Search: 6.15 sec
Data:  100000 points, Queries:    0 => Build: 29.67 sec, Search: 0.00 sec
Data:   10000 points, Queries:  100 => Build: 4.10 sec, Search: 11.00 sec
Data:  100000 points, Queries:  100 => Build: 31.94 sec, Search: 17.57 sec
Data: 1000000 points, Queries:   20 => Build: 367.44 sec, Search: 2.93 sec
Data:  100000 points, Queries:  500 => Build: 36.43 sec, Search: 48.58 sec
Data:  100000 points, Queries:    0 => Build: 40.72 sec, Search: 0.00 sec
Data:       0 points, Queries:   20 => Build: 0.00 sec, Search: 0.28 sec
--------------------------------------------

=== Summary of VP MWV Tree Tests ===

Total Build Time: 538.64 sec, Total Search Time: 92.43 sec
------------------------------------------

**-----VP_MWV Run 2-------------**

In [ ]:
results = []
st_idx = 0  # reset the dataset slicing index for processing test cases

vptree = VPTree()

for idx, (data_size, query_size) in enumerate(test_combinations):
    print(f"\n{idx+1}: === Testing with Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # Slice the current data chunk for this test.
    end_idx = st_idx + data_size
    current_data = combined_dataset[st_idx:end_idx]
    st_idx = end_idx

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[idx]

    # --- Build the VP tree ---
    build_start = time.perf_counter()
    for pt in tqdm(current_data):
        vptree.insert(pt)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc="Querying"):
        search_start = time.perf_counter()
        r_min, nn_pt = vptree.nearest_neighbor(query=query)
        search_time += time.perf_counter() - search_start

    results.append(((data_size, query_size), build_time, search_time))
    print(f"Build Time: {build_time:.2f} sec, Search Time: {search_time:.2f} sec")

# ----------------------------
# Summary of Results
# ----------------------------
print("\n=== Summary of Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    print(f"Data: {data_size:>7} points, Queries: {query_size:>4} => Build: {b_time:.2f} sec, Search: {s_time:.2f} sec")

print("--------------------------------------------")
# ----------------------------
# Total Results
# ----------------------------
build_time_total = search_time_total = 0.0
print("\n=== Summary of VP MWV Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    build_time_total += b_time
    search_time_total += s_time
print(f"\nTotal Build Time: {build_time_total:.2f} sec, Total Search Time: {search_time_total:.2f} sec")

print("--------------------------------------------")
total_time = build_time_total + search_time_total
print(f"Total Time: {total_time:.2f}")


1: === Testing with Data Size: 10,000 points, Query Size: 0 queries ===


100%|██████████| 10000/10000 [00:01<00:00, 5332.41it/s]
Querying: 0it [00:00, ?it/s]


Build Time: 1.88 sec, Search Time: 0.00 sec

2: === Testing with Data Size: 100,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:05<00:00, 19.76it/s]


Build Time: 26.15 sec, Search Time: 5.03 sec

3: === Testing with Data Size: 0 points, Query Size: 100 queries ===


...

Build Time: 38.96 sec, Search Time: 36.09 sec

9: === Testing with Data Size: 100,000 points, Query Size: 0 queries ===


100%|██████████| 100000/100000 [00:38<00:00, 2578.87it/s]
Querying: 0it [00:00, ?it/s]


Build Time: 38.78 sec, Search Time: 0.00 sec

10: === Testing with Data Size: 0 points, Query Size: 20 queries ===


0it [00:00, ?it/s]
Querying: 100%|██████████| 20/20 [00:00<00:00, 68.66it/s]

Build Time: 0.00 sec, Search Time: 0.29 sec

=== Summary of Tree Tests ===
Data:   10000 points, Queries:    0 => Build: 1.88 sec, Search: 0.00 sec
Data:  100000 points, Queries:  100 => Build: 26.15 sec, Search: 5.03 sec
Data:       0 points, Queries:  100 => Build: 0.00 sec, Search: 4.96 sec
Data:  100000 points, Queries:    0 => Build: 27.73 sec, Search: 0.00 sec
Data:   10000 points, Queries:  100 => Build: 3.76 sec, Search: 10.17 sec
Data:  100000 points, Queries:  100 => Build: 32.69 sec, Search: 13.95 sec
Data: 1000000 points, Queries:   20 => Build: 354.47 sec, Search: 0.86 sec
Data:  100000 points, Queries:  500 => Build: 38.96 sec, Search: 36.09 sec
Data:  100000 points, Queries:    0 => Build: 38.78 sec, Search: 0.00 sec
Data:       0 points, Queries:   20 => Build: 0.00 sec, Search: 0.29 sec
--------------------------------------------

=== Summary of VP MWV Tree Tests ===

Total Build Time: 524.42 sec, Total Search Time: 71.35 sec
------------------------------------------

**--------VP MWV Analysis---------**

In [ ]:
import re
import numpy as np

# The result strings
results = [
    'Total Time: 631.07',
    'Total Time: 595.78'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time:\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 613.4250 sec, Std Dev: 24.9538 sec, CV: 4.07%


-----------------------------------------------

### Exhaustive
---

In [ ]:
def euclidean_distance(a, b):
    diff = a - b
    return np.sqrt(np.dot(diff, diff))

In [ ]:
def exhaustive_search(query, dataset, dist_fn):
    best_dist = float('inf')
    best_point = None
    for pt in dataset:
        d = dist_fn(query, pt)
        if d < best_dist:
            best_dist = d
            best_point = pt
    return best_dist, best_point

------------------------------

**-----------Exhuastive Run 1------------**

In [ ]:
results = []
st_idx = 0
accumulated_data = []

for idx, (data_size, query_size) in enumerate(test_combinations):
    print(f"\n{idx+1}: === Data Size: {data_size:,}, Query Size: {query_size:,} ===")

    # Add new data if any
    end_idx = st_idx + data_size
    current_data = combined_dataset[st_idx:end_idx]
    st_idx = end_idx

    if data_size > 0:
        accumulated_data.extend(current_data)

    if not accumulated_data:
        print("Skipping search due to empty dataset.")
        results.append(((data_size, query_size), 0.0, 0.0))
        continue

    queries = queries_list[idx]

    build_start = time.perf_counter()
    accumulated_array = np.array(accumulated_data)
    build_time = time.perf_counter() - build_start


    search_time = 0.0

    for query in tqdm(queries, desc="Querying"):
        start = time.perf_counter()
        _, _ = exhaustive_search(query, accumulated_array, euclidean_distance)
        search_time += time.perf_counter() - start

    results.append(((data_size, query_size), build_time, search_time))

    print(f"Search Time: {search_time:.2f} sec")

# ----------------------------
# Exhaustive Search Summary
# ----------------------------
build_time_total = sum(b_time for (_, _), b_time, _ in results)
search_time_total = sum(s_time for (_, _), _, s_time in results)

print("---------------------------")
print(f"Total Time: {build_time_total + search_time_total:.2f} sec")


1: === Data Size: 10,000, Query Size: 0 ===


Querying: 0it [00:00, ?it/s]


Search Time: 0.00 sec

2: === Data Size: 100,000, Query Size: 100 ===


Querying: 100%|██████████| 100/100 [00:25<00:00,  3.90it/s]


Search Time: 25.55 sec

3: === Data Size: 0, Query Size: 100 ===


...

Search Time: 1606.77 sec

9: === Data Size: 100,000, Query Size: 0 ===


Querying: 0it [00:00, ?it/s]


Search Time: 0.00 sec

10: === Data Size: 0, Query Size: 20 ===


Querying: 100%|██████████| 20/20 [01:08<00:00,  3.41s/it]

Search Time: 68.10 sec
---------------------------
Total Time: 1913.08 sec


**-------Exhaustive Run 2--------------**

In [ ]:
results = []
st_idx = 0
accumulated_data = []

for idx, (data_size, query_size) in enumerate(test_combinations):
    print(f"\n{idx+1}: === Data Size: {data_size:,}, Query Size: {query_size:,} ===")

    # Add new data if any
    end_idx = st_idx + data_size
    current_data = combined_dataset[st_idx:end_idx]
    st_idx = end_idx

    if data_size > 0:
        accumulated_data.extend(current_data)

    if not accumulated_data:
        print("Skipping search due to empty dataset.")
        results.append(((data_size, query_size), 0.0, 0.0))
        continue

    queries = queries_list[idx]

    build_start = time.perf_counter()
    accumulated_array = np.array(accumulated_data)
    build_time = time.perf_counter() - build_start


    search_time = 0.0

    for query in tqdm(queries, desc="Querying"):
        start = time.perf_counter()
        _, _ = exhaustive_search(query, accumulated_array, euclidean_distance)
        search_time += time.perf_counter() - start

    results.append(((data_size, query_size), build_time, search_time))

    print(f"Search Time: {search_time:.2f} sec")

# ----------------------------
# Exhaustive Search Summary
# ----------------------------
build_time_total = sum(b_time for (_, _), b_time, _ in results)
search_time_total = sum(s_time for (_, _), _, s_time in results)

print("---------------------------")
print(f"Total Time: {build_time_total + search_time_total:.2f} sec")


1: === Data Size: 10,000, Query Size: 0 ===


Querying: 0it [00:00, ?it/s]


Search Time: 0.00 sec

2: === Data Size: 100,000, Query Size: 100 ===


Querying: 100%|██████████| 100/100 [00:25<00:00,  3.93it/s]


Search Time: 25.36 sec

3: === Data Size: 0, Query Size: 100 ===


...

Search Time: 1599.79 sec

9: === Data Size: 100,000, Query Size: 0 ===


Querying: 0it [00:00, ?it/s]


Search Time: 0.00 sec

10: === Data Size: 0, Query Size: 20 ===


Querying: 100%|██████████| 20/20 [01:08<00:00,  3.42s/it]

Search Time: 68.43 sec
---------------------------
Total Time: 1901.83 sec


**-------Exhaustive Search Analysis----------**

In [ ]:
import re
import numpy as np

# The result strings
results = [
    'Total Time: 1913.08',
    'Total Time: 1901.83'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time:\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)

Total Time - Mean: 1907.4550 sec, Std Dev: 7.9550 sec, CV: 0.42%


-------------------------------------------

**----------QuickSelect Algorithm-----------**

In [12]:
from collections import deque
from typing import List, Tuple

#*-----------------------------------------------------------------------------
def quickselect(distances:List, k:int):
    """QuickSelect algorithm to find the k-th smallest distance."""

    if len(distances) == 1:
        return distances[0]

    if len(distances) == 0:
        return None

    stack = deque()
    stack.append( (distances, k) )
    while stack:

        distances, k = stack.pop()
        if len(distances) == 1:
            return distances[0]

        # Returned as a tuple
        pivot = median_of_medians(distances)
        lows, highs, pivots = partition(distances, pivot)

        if len(lows) > k:
            stack.append((lows, k))
        elif len(lows) + len(pivots) > k:
            return pivot
        else:
            stack.append((highs, k - len(lows) - len(pivots)))
    return pivot

#*-----------------------------------------------------------------------------
def partition(distances, pivot):
    lows, highs, pivots = [], [], []

    for d in distances:
        if d[0] < pivot[0]:
            lows.append(d)
        elif d[0] > pivot[0]:
            highs.append(d)
        else:
            pivots.append(d)
    return lows, highs, pivots

#*-----------------------------------------------------------------------------
def median_of_medians(distances, grp_size=5):
    """Find the index of the median distance in a list of distances.
    Returns the index and the median distance as a tuple."""

    stack = deque()
    # Split the list in chunks of grp_size
    # And add the lists to the stack on the right side
    for i in range(0, len(distances), grp_size):
        subList = distances[i:i+grp_size]
        insertion_sort(subList)
        stack.append(subList)

    # We want to find the median of the medians
    # and return the index of that median
    median_list = []
    stack_count = len(stack)
    while 0 < stack_count:

        subList = stack.pop()
        stack_count -= 1

        mid_index = len(subList) // 2
        median_list.append(subList[mid_index])

        if len(median_list) == grp_size:
            insertion_sort(median_list)
            # append to the left, since we are popping from the right
            stack.appendleft(median_list)
            if stack_count == 0:
                stack_count = len(stack)
            median_list = []
        # If the count is 0, we have reached the end of the old stack
        # and need to insert the remaining elements,
        # stack count and the median list will be reset
        if stack_count == 0 and len(median_list) > 0:
            if len(median_list) > 1:
                insertion_sort(median_list)
            stack.appendleft(median_list)
            stack_count = len(stack)
            if stack_count == 1 and len(median_list) == 1:
                return median_list[0]
            median_list = []

#*-----------------------------------------------------------------------------
def insertion_sort(arr):
# In-place insertion sort, efficient for small arrays
    for i in range(1, len(arr)):
        key = arr[i]
        j = i - 1
        while j >= 0 and arr[j][0] > key[0]:
            arr[j + 1] = arr[j]
            j -= 1
        arr[j + 1] = key
#*-----------------------------------------------------------------------------

----------------------------------------

**-------VP Dynamic-------**

In [13]:
import numpy as np
from collections import deque
#*-----------------------------------------------------------------------------

class VPNode:
    def __init__(self, point=None):
        self.point = point
        self.mu = 0
        self.threshold_points = []
        self.left_tr = None
        self.right_tr = None
        self.points = []
        self.centroid = None
        self.radius = 0

        # Cache for numpy arrays
        self._np_points = None
        self._np_threshold_points = None
#*-----------------------------------------------------------------------------

class VPTree:
    def __init__(self, leaf_size=50, epsilon=0.1):
        self.root = None
        self.leaf_size = leaf_size
        self.epsilon = epsilon

    #*-----------------------------------------------------------------------------
    def build(self, points):
        self.root = VPNode()
        stack = deque([(self.root, points)])

        while stack:
            node, pts = stack.pop()
            n_pts = len(pts)

            if n_pts <= self.leaf_size:
                node.points = [p for p in pts]
                node.centroid = np.mean(pts, axis=0)
                continue

            node.point = pts[0]
            remaining_pts = pts[1:]

            if len(remaining_pts) == 0:
                node.points = [node.point]
                node.centroid = node.point
                continue

            # distances for this node (exact band semantics)
            diff       = remaining_pts - node.point                 # (m, D)
            distances  = np.linalg.norm(diff, axis=1)               # (m,)

            # median via quickselect on (value, idx) tuples
            coords     = [(float(distances[i]), i) for i in range(distances.size)]
            k          = len(coords) // 2
            mu, _      = quickselect(coords, k)
            node.mu    = float(mu)

            # band: exact |d - mu| < epsilon
            dev        = np.abs(distances - node.mu)
            band_mask  = dev < self.epsilon
            thr_idx    = np.flatnonzero(band_mask)

            # choose up to leaf_size closest to mu (no full sort: argpartition on dev)
            if thr_idx.size > 0:
                if thr_idx.size > self.leaf_size:
                    dev_band = dev[thr_idx]
                    sel      = np.argpartition(dev_band, self.leaf_size - 1)[:self.leaf_size]
                    keep_idx = thr_idx[sel]
                else:
                    keep_idx = thr_idx

                # store threshold points
                thr_arr = remaining_pts[keep_idx]                   # ndarray for math
                node.threshold_points = [p for p in thr_arr]        # list if appending later

                # centroid & radius over stored band points only
                c = thr_arr.mean(axis=0)
                node.centroid = c

                # children masks: everything else, split around mu
                keep_mask  = np.zeros_like(distances, dtype=bool)
                keep_mask[keep_idx] = True
                inner_mask = (distances <= node.mu) & (~keep_mask)
                outer_mask = (distances >  node.mu) & (~keep_mask)
            else:
                # no threshold storage; pure binary split
                inner_mask = (distances <= node.mu)
                outer_mask = (distances >  node.mu)

            inner_pts = remaining_pts[inner_mask]
            outer_pts = remaining_pts[outer_mask]


            if len(inner_pts) > 0:
                node.left_tr = VPNode()
                stack.appendleft((node.left_tr, inner_pts))

            if len(outer_pts) > 0:
                node.right_tr = VPNode()
                stack.appendleft((node.right_tr, outer_pts))

        return self.root

    #*-----------------------------------------------------------------------------
    def prepare_for_search(self):
        """
        Convert all list-based collections to numpy arrays for efficient search.
        This should be called once after all insertions and before searches.
        Also computes exact radius for each node's stored points.
        """
        stack = [self.root]
        while stack:
            node = stack.pop()
            if node is None:
                continue

            # Convert and compute exact radius for regular points
            if node.points:
                node._np_points = np.array(node.points)
                if len(node.points) > 1:
                    diffs = node._np_points - node.centroid
                    node.radius = float(np.sqrt(np.sum(diffs * diffs, axis=1)).max())
                else:
                    node.radius = 0.0

            # Convert and compute exact radius for threshold points
            if node.threshold_points:
                node._np_threshold_points = np.array(node.threshold_points)
                if len(node.threshold_points) > 1:
                    diffs = node._np_threshold_points - node.centroid
                    node.radius = float(np.sqrt(np.sum(diffs * diffs, axis=1)).max())
                else:
                    node.radius = 0.0

            # Add child nodes to stack
            if node.left_tr:
                stack.append(node.left_tr)
            if node.right_tr:
                stack.append(node.right_tr)

    #*-----------------------------------------------------------------------------
    def search(self, query):
        """
        Nearest-neighbor search for the VP-tree with per-node threshold micro-leaves.
        Assumes `prepare_for_search()` has been called so that `_np_points` and
        `_np_threshold_points` are either NumPy arrays or None.
        """
        best_dist = float("inf")
        best_point = None
        stack = [self.root]

        while stack:
            node = stack.pop()
            if node is None:
                continue

            # --- Leaf block (node.points is non-empty) ---
            if node.points:
                if node.centroid is not None:
                    dc = np.linalg.norm(query - node.centroid)
                    if dc - node.radius >= best_dist:
                        continue

                P = node._np_points
                if P is not None:
                    dists = np.linalg.norm(P - query, axis=1)
                    k = int(np.argmin(dists))
                    dk = dists[k]
                    if dk < best_dist:
                        best_dist = dk
                        best_point = P[k]
                continue

            # --- Vantage point at this internal node ---
            dv = None
            if node.point is not None:
                dv = np.linalg.norm(node.point - query)
                if dv < best_dist:
                    best_dist = dv
                    best_point = node.point

            # --- Threshold micro-leaf at this node (execute iff present) ---
            if node.threshold_points:  # no extra array allocations
                # prune using its bounding ball
                dc = np.linalg.norm(query - node.centroid)
                if dc - node.radius < best_dist:
                    T = node._np_threshold_points
                    if T is not None:
                        td = np.linalg.norm(T - query, axis=1)
                        k = int(np.argmin(td))
                        tk = td[k]
                        if tk < best_dist:
                            best_dist = tk
                            best_point = T[k]

            # --- Subtree routing: standard VP rule (no epsilon in routing) ---
            if dv is not None:
                if dv <= node.mu:
                    near, far = node.left_tr, node.right_tr
                else:
                    near, far = node.right_tr, node.left_tr

                # Push far first so near is popped first (LIFO)
                if far is not None and abs(dv - node.mu) <= best_dist:
                    stack.append(far)
                if near is not None:
                    stack.append(near)

        return best_dist, best_point

    #*-----------------------------------------------------------------------------
    def insert(self, point):
        """
        Insert a new point into the VP-Tree with fast list-based operations.
        """

        if self.root is None:
            self.root = VPNode()
            self.root.points = [point]
            self.root.centroid = point
            self.root.radius = 0
            return

        current = self.root

        while not current.points:
            if current.point is None:
                if current.left_tr:
                    current = current.left_tr
                elif current.right_tr:
                    current = current.right_tr
                else:
                    current.points = [point]
                    current.centroid = point
                    current.radius = 0
                    return
            else:
                dist_to_vp = np.linalg.norm(point - current.point)

                if abs(dist_to_vp - current.mu) <= self.epsilon:
                    if not current.threshold_points:
                        current.threshold_points = [point]
                        current.centroid = point
                        current.radius = 0
                        return
                    elif len(current.threshold_points) < self.leaf_size:
                        n = len(current.threshold_points)
                        c_old = current.centroid
                        c_new = (n * c_old + point) / (n + 1)

                        current.threshold_points.append(point)
                        current.centroid = c_new
                        return


                if dist_to_vp < current.mu - self.epsilon:
                    if current.left_tr:
                        current = current.left_tr
                    else:
                        current.left_tr = VPNode()
                        current.left_tr.points = [point]
                        current.left_tr.centroid = point
                        current.left_tr.radius = 0
                        return
                else:
                    if current.right_tr:
                        current = current.right_tr
                    else:
                        current.right_tr = VPNode()
                        current.right_tr.points = [point]
                        current.right_tr.centroid = point
                        current.right_tr.radius = 0
                        return

        n = len(current.points)

        # Dynamically adjust the node size for homogeneous point storage
        if n == 2 * self.leaf_size:
            self.leaf_size = n

        c_old = current.centroid
        c_new = (n * c_old + point) / (n + 1)

        current.points.append(point)
        current.centroid = c_new
#*------------------------------------------------------------------------------------------

-------------------------------------------------

### Log Ratio
---

**---------------Run 1--------------**

In [14]:
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 8600
epsilon_rebuild = 0.02
epsilon_insert = 0.7

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

# prepare for search sentinel
must_prepare = False

for data_size, query_load in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
        # after rebuild path
        if points_at_last_rebuild > 0:   # we actually built a tree
            must_prepare = True
        else:
            must_prepare = False  # nothing to prepare yet
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        if batch_size > 0:
          must_prepare = True

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    if vptree is not None and query_load > 0:
      if must_prepare:
          vptree.prepare_for_search()
          must_prepare = False
      for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
          r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding VP Tree from scratch with 10,000 points.
Build Time: 0.0125 sec
Total Search Time: 0.000001 sec

Batch 2: Adding 100,000 points
Rebuilding VP Tree from scratch with 110,000 points.
Build Time: 0.5328 sec


Querying batch 2: 100%|██████████| 100/100 [00:00<00:00, 333.51it/s]


Total Search Time: 0.340879 sec

Batch 3: Adding 0 points


Inserting 0 points: 0it [00:00, ?it/s]


Insert Time: 0.0030 sec


...

Insert Time: 2.8662 sec
Total Search Time: 0.000002 sec

Batch 10: Adding 0 points


Inserting 0 points: 0it [00:00, ?it/s]


Insert Time: 0.0022 sec


Querying batch 10: 100%|██████████| 20/20 [00:00<00:00, 291.70it/s]

Total Search Time: 0.492020 sec

=== Summary of VP Tree Scaling Test ===
Points: 10,000, Build Time: 0.0125 sec, Total Search Time: 0.000001 sec, Total Time: 0.0125 sec
Points: 110,000, Build Time: 0.5328 sec, Total Search Time: 0.340879 sec, Total Time: 0.8737 sec
Points: 110,000, Insert Time: 0.0030 sec, Total Search Time: 0.298077 sec, Total Time: 0.3011 sec
Points: 210,000, Insert Time: 1.5705 sec, Total Search Time: 0.000002 sec, Total Time: 1.5705 sec
Points: 220,000, Insert Time: 0.1587 sec, Total Search Time: 0.755409 sec, Total Time: 0.9141 sec
Points: 320,000, Insert Time: 1.5500 sec, Total Search Time: 0.961374 sec, Total Time: 2.5114 sec
Points: 1,320,000, Insert Time: 15.2114 sec, Total Search Time: 1.592070 sec, Total Time: 16.8034 sec
Points: 1,420,000, Build Time: 17.3968 sec, Total Search Time: 2.330490 sec, Total Time: 19.7273 sec
Points: 1,520,000, Insert Time: 2.8662 sec, Total Search Time: 0.000002 sec, Total Time: 2.8662 sec
Points: 1,520,000, Insert Time: 0.0022 

**---------Run 2-----------**

In [34]:
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 8600
epsilon_rebuild = 0.02
epsilon_insert = 0.7

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

# prepare for search sentinel
must_prepare = False

for data_size, query_load in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
        # after rebuild path
        if points_at_last_rebuild > 0:   # we actually built a tree
            must_prepare = True
        else:
            must_prepare = False  # nothing to prepare yet
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        if batch_size > 0:
          must_prepare = True

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    if vptree is not None and query_load > 0:
      if must_prepare:
          vptree.prepare_for_search()
          must_prepare = False
      for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
          r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding VP Tree from scratch with 10,000 points.
Build Time: 0.0159 sec
Total Search Time: 0.000001 sec

Batch 2: Adding 100,000 points
Rebuilding VP Tree from scratch with 110,000 points.
Build Time: 0.6328 sec


Querying batch 2: 100%|██████████| 100/100 [00:00<00:00, 258.57it/s]


Total Search Time: 0.417573 sec

Batch 3: Adding 0 points


Inserting 0 points: 0it [00:00, ?it/s]


Insert Time: 0.0032 sec


...

Insert Time: 2.7688 sec
Total Search Time: 0.000002 sec

Batch 10: Adding 0 points


Inserting 0 points: 0it [00:00, ?it/s]


Insert Time: 0.0025 sec


Querying batch 10: 100%|██████████| 20/20 [00:00<00:00, 256.94it/s]

Total Search Time: 0.501011 sec

=== Summary of VP Tree Scaling Test ===
Points: 10,000, Build Time: 0.0159 sec, Total Search Time: 0.000001 sec, Total Time: 0.0159 sec
Points: 110,000, Build Time: 0.6328 sec, Total Search Time: 0.417573 sec, Total Time: 1.0504 sec
Points: 110,000, Insert Time: 0.0032 sec, Total Search Time: 0.352250 sec, Total Time: 0.3555 sec
Points: 210,000, Insert Time: 1.4921 sec, Total Search Time: 0.000002 sec, Total Time: 1.4921 sec
Points: 220,000, Insert Time: 0.1545 sec, Total Search Time: 0.725278 sec, Total Time: 0.8798 sec
Points: 320,000, Insert Time: 1.4885 sec, Total Search Time: 1.090443 sec, Total Time: 2.5790 sec
Points: 1,320,000, Insert Time: 15.0516 sec, Total Search Time: 1.557295 sec, Total Time: 16.6089 sec
Points: 1,420,000, Build Time: 19.4474 sec, Total Search Time: 2.411869 sec, Total Time: 21.8593 sec
Points: 1,520,000, Insert Time: 2.7688 sec, Total Search Time: 0.000002 sec, Total Time: 2.7688 sec
Points: 1,520,000, Insert Time: 0.0025 

**---------------Run 3---------------**

In [38]:
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 8600
epsilon_rebuild = 0.02
epsilon_insert = 0.7

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

# prepare for search sentinel
must_prepare = False

for data_size, query_load in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
        # after rebuild path
        if points_at_last_rebuild > 0:   # we actually built a tree
            must_prepare = True
        else:
            must_prepare = False  # nothing to prepare yet
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        if batch_size > 0:
          must_prepare = True

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    if vptree is not None and query_load > 0:
      if must_prepare:
          vptree.prepare_for_search()
          must_prepare = False
      for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
          r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 10,000 points
Rebuilding VP Tree from scratch with 10,000 points.
Build Time: 0.0861 sec
Total Search Time: 0.000001 sec

Batch 2: Adding 100,000 points
Rebuilding VP Tree from scratch with 110,000 points.
Build Time: 0.5518 sec


Querying batch 2: 100%|██████████| 100/100 [00:00<00:00, 276.06it/s]


Total Search Time: 0.394341 sec

Batch 3: Adding 0 points


Inserting 0 points: 0it [00:00, ?it/s]


Insert Time: 0.0030 sec


...

Insert Time: 2.7502 sec
Total Search Time: 0.000002 sec

Batch 10: Adding 0 points


Inserting 0 points: 0it [00:00, ?it/s]


Insert Time: 0.0019 sec


Querying batch 10: 100%|██████████| 20/20 [00:00<00:00, 265.65it/s]

Total Search Time: 0.484216 sec

=== Summary of VP Tree Scaling Test ===
Points: 10,000, Build Time: 0.0861 sec, Total Search Time: 0.000001 sec, Total Time: 0.0861 sec
Points: 110,000, Build Time: 0.5518 sec, Total Search Time: 0.394341 sec, Total Time: 0.9462 sec
Points: 110,000, Insert Time: 0.0030 sec, Total Search Time: 0.373384 sec, Total Time: 0.3763 sec
Points: 210,000, Insert Time: 1.5470 sec, Total Search Time: 0.000002 sec, Total Time: 1.5470 sec
Points: 220,000, Insert Time: 0.1538 sec, Total Search Time: 0.735600 sec, Total Time: 0.8894 sec
Points: 320,000, Insert Time: 1.5139 sec, Total Search Time: 1.152398 sec, Total Time: 2.6663 sec
Points: 1,320,000, Insert Time: 15.1062 sec, Total Search Time: 1.546848 sec, Total Time: 16.6530 sec
Points: 1,420,000, Build Time: 19.2494 sec, Total Search Time: 2.327384 sec, Total Time: 21.5767 sec
Points: 1,520,000, Insert Time: 2.7502 sec, Total Search Time: 0.000002 sec, Total Time: 2.7502 sec
Points: 1,520,000, Insert Time: 0.0019 

**--------Log Threshold Analysis--------------**

In [39]:
import re
import numpy as np

# The result strings
results = [
    'Total Time:: 46.0745',
    'Total Time:: 48.1131',
    'Total Time:: 47.9775'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)

Total Time - Mean: 47.3884 sec, Std Dev: 1.1399 sec, CV: 2.41%


--------------------------------------------------

### Evolutionary Algorithm Lamarckian
---

In [15]:
import math
import random
from dataclasses import dataclass
from typing import Dict, List, Optional
from collections import deque
import numpy as np

# ------------------------------- robust stats ---------------------------------
def _mad(x: List[float]) -> float:
    if len(x) == 0:
        return 0.0
    x_arr = np.asarray(x, float)
    m = float(np.median(x_arr))
    return 1.4826 * float(np.median(np.abs(x_arr - m)))

def _winsor_logmean(log_values: List[float], delta0: float = 0.05, anytime: bool = True) -> float:
    if not log_values:
        return 1.0
    y = np.asarray(log_values, float)
    n = y.size
    if n == 1:
        return float(np.exp(y.item()))
    med = float(np.median(y))
    mad = _mad(y)
    if mad == 0.0:
        return float(np.exp(med))
    delta_n = (delta0 / (n*n)) if anytime else (1.0 / n)
    delta_n = min(max(delta_n, 1e-12), 1.0 - 1e-12)
    kappa = float(np.sqrt(2.0 * np.log(1.0 / delta_n)))
    lo, hi = med - kappa * mad, med + kappa * mad
    clipped = np.clip(y, lo, hi)
    return float(np.exp(np.mean(clipped)))

# ====================== EA individual (diagnostics + policy) ===================
@dataclass
class _Ind:
    c_pr: float
    c_pi: float
    c_qr: float
    c_qi: float
    fitness: float = 1.0
    experience: int = 0
    total_time: float = 0.0
    # keep a short history of log-losses for robust fitness
    _logloss_hist: deque = None

    def __post_init__(self):
        if self._logloss_hist is None:
            self._logloss_hist = deque(maxlen=64)

    def update_fit(self, loss: float):
        """
        Use the SAME robustification paradigm as elsewhere: winsorized log-mean.
        No extra hyperparameters beyond delta0 already used in _winsor_logmean.
        """
        self.experience += 1

        # Guard and record log-loss
        loss = float(loss)
        if not math.isfinite(loss) or loss <= 0.0:
            loss = 1.0
        self._logloss_hist.append(math.log(loss))

        # Robust geometric mean of loss (Catoni/winsor), then invert to get fitness
        gmean_loss = _winsor_logmean(list(self._logloss_hist), delta0=0.05, anytime=True)
        if not math.isfinite(gmean_loss) or gmean_loss <= 0.0:
            gmean_loss = 1.0

        # Optional: keep total_time for backwards-compatibility of any diagnostics
        self.total_time += loss

        # Higher fitness for lower (robust) loss
        self.fitness = 1.0 / max(gmean_loss, 1e-9)

# ===================== Predictor/selector with EA + Lamarckian =================
class EvolutionaryOperationSelector:
    """
    Deterministic analytic predictor + robust multiplicative updates,
    augmented with:
      • Population-as-policy (soft): blend sampled individual's (c_*) with live scalers for prediction.
      • Lamarckian coupling: write back the live scalers into elite genes softly each update.

    Decision logic remains the same inequality; only the scalers fed into the predictor get a
    small, controlled EA blend to (hopefully) reduce prediction AbsErr.
    """

    G = {"c_pr": (0.85, 1.15), "c_pi": (0.85, 1.15), "c_qr": (0.85, 1.15), "c_qi": (0.85, 1.15)}

    def __init__(self,
                 leaf_size: int = 8100,
                 min_gap_batches: int = 1,
                 population_size: int = 20,
                 rebuild_margin: float = 1.0,
                 seed: Optional[int] = None,
                 c_pr_init: float = 0.1,
                 # EA knobs
                 policy_mix: float = 0.25,          # blend weight toward sampled individual
                 alpha_loss: float = 0.7,           # weight on measured/pred ratio in EA loss
                 wrong_arm_margin: float = 0.05,    # margin to flag "wrong arm" in hindsight
                 evolve_every: int = 3,             # evolve after this many decisions
                 # Lamarckian knobs:
                 lamarck_lambda: float = 0.85,      # elite := lambda * elite + (1-lambda)*live_scalers
                 lamarck_every: int = 1):           # apply every k updates
        if seed is not None:
            random.seed(seed); np.random.seed(seed)

        self.cpr_floor_min = float(c_pr_init)

        self.leaf_size = max(1, int(leaf_size))
        self._min_gap_batches = max(1, int(min_gap_batches))
        self._rebuild_margin = float(max(1e-12, rebuild_margin))

        # geometry state
        self._n_at_rebuild: int = 0
        self._accum_since_rebuild: int = 0
        self._last_was_rebuild: bool = False
        self._since_rebuild_batches: int = 10**9

        # epoch bookkeeping
        self._epoch_id: int = 0
        self._tpi_status: str = "unknown"
        self._tpqI_status: str = "unknown"

        # histories (optional)
        self._reb_times: List[float] = []
        self._ins_times: List[float] = []
        self.recent_query_times: deque = deque(maxlen=64)

        # EA population
        self.population_size = max(8, int(population_size))
        self.population: List[_Ind] = []
        self.elite_ratio = 0.35
        self.base_mut = 0.12
        self.total_decisions = 0
        self._ensure_pop()

        # bases (exact when set)
        self.tpr: Optional[float]  = None
        self.tpqR: Optional[float] = None
        self.tpi: Optional[float]  = None
        self.tpqI: Optional[float] = None

        # scalers (learned by the robust updates)
        self._c_pr: float = max(self.cpr_floor_min, float(c_pr_init))
        self._c_qr: float = 1.0
        self._c_pi: float = 1.0
        self._c_qi: float = 1.0

        # predictor snapshots
        self.last_prediction: Optional[Dict] = None   # (kept as before; policy-blended)
        #  store what policy used to compute last decision:
        self._last_policy: Optional[Dict] = None      # {"ind": _Ind, "c_used": dict, "pred": dict, "act": "rebuild"/"insert"}

        # multiplicative learning knobs
        self._alpha_base = 0.20
        self._alpha_cat  = 0.10

        # per-component log-ratio histories
        self._hist_logs = {
            "reb_q":  deque(maxlen=128),
            "ins_op": deque(maxlen=128),
            "ins_q":  deque(maxlen=128),
        }

        # deterministic query velocity state
        self.vq_anchor: Optional[float] = None
        self.ns_at_anchor: Optional[float] = None

        #  EA control params
        self.policy_mix = float(min(max(policy_mix, 0.0), 1.0))
        self.alpha_loss = float(min(max(alpha_loss, 0.0), 1.0))
        self.wrong_arm_margin = float(max(0.0, wrong_arm_margin))
        self.evolve_every = max(1, int(evolve_every))
        self._decisions_since_evolve = 0

        #  Lamarckian params
        self.lamarck_lambda = float(min(max(lamarck_lambda, 0.0), 1.0))
        self.lamarck_every = max(1, int(lamarck_every))
        self._updates_since_lamarck = 0

    # ---------------------------- EA utilities -----------------------------
    def _ensure_pop(self):
        if self.population: return
        g = self.G
        def r(lo, hi): return lo + (hi - lo) * random.random()
        for _ in range(self.population_size):
            cpr = r(*g["c_pr"])
            cpr = max(self.cpr_floor_min, cpr)  # obey floor
            self.population.append(_Ind(cpr, r(*g["c_pi"]), r(*g["c_qr"]), r(*g["c_qi"])))

    def _w(self, ind: _Ind) -> float:
        return ind.fitness * math.sqrt(ind.experience + 1.0)

    def _mut_rate(self) -> float:
        above = sum(self._w(i) for i in self.population if i.c_pr > 1 and i.c_pi > 1 and i.c_qr > 1 and i.c_qi > 1)
        below = sum(self._w(i) for i in self.population) - above
        denom = max(1e-9, above + below)
        p_a = above / denom; p_b = below / denom
        ent = 0.0
        if p_a > 0: ent -= p_a * math.log2(p_a)
        if p_b > 0: ent -= p_b * math.log2(p_b)
        ent = ent / math.log2(2)
        return max(0.02, min(0.35, self.base_mut * (1.2 - 0.4 * ent)))

    def _mut_gene(self, v, lo, hi, rate):
        if random.random() >= rate: return v
        span = hi - lo
        step = 0.18 * span
        return float(min(hi, max(lo, v + random.uniform(-step, step))))

    def _clone(self, ind: _Ind) -> _Ind:
        return _Ind(ind.c_pr, ind.c_pi, ind.c_qr, ind.c_qi, ind.fitness, ind.experience, ind.total_time)

    def _cross(self, a: _Ind, b: _Ind) -> _Ind:
        return _Ind((a.c_pr + b.c_pr)/2.0, (a.c_pi + b.c_pi)/2.0, (a.c_qr + b.c_qr)/2.0, (a.c_qi + b.c_qi)/2.0)

    def _evolve(self):
        if len(self.population) < 6: return
        pop = sorted(self.population, key=self._w, reverse=True)
        elites = pop[:max(1, int(self.elite_ratio * len(pop)))]
        rate = self._mut_rate()
        newp = [self._clone(e) for e in elites]
        g = self.G
        while len(newp) < self.population_size:
            a, b = random.sample(elites, 2) if len(elites) >= 2 else (random.choice(pop), random.choice(pop))
            child = self._cross(a, b)
            child.c_pr = max(self.cpr_floor_min, self._mut_gene(child.c_pr, *g["c_pr"], rate))
            child.c_qr = self._mut_gene(child.c_qr, *g["c_qr"], rate)
            child.c_pi = self._mut_gene(child.c_pi, *g["c_pi"], rate)
            child.c_qi = self._mut_gene(child.c_qi, *g["c_qi"], rate)
            newp.append(child)
        self.population = newp

    #  sample individual is proportional with weight
    def _sample_individual(self) -> _Ind:
        if not self.population:
            self._ensure_pop()
        weights = [max(1e-12, self._w(ind)) for ind in self.population]
        total = sum(weights)
        r = random.random() * total
        acc = 0.0
        for ind, w in zip(self.population, weights):
            acc += w
            if acc >= r:
                return ind
        return random.choice(self.population)

    #  blend live scalers with genes (soft policy)
    def _blend_scalers(self, ind: _Ind, mix: float) -> Dict[str, float]:
        mix = float(min(max(mix, 0.0), 1.0))
        base = {"c_pr": self._c_pr, "c_pi": self._c_pi, "c_qr": self._c_qr, "c_qi": self._c_qi}
        gene = {"c_pr": max(self.cpr_floor_min, ind.c_pr), "c_pi": ind.c_pi, "c_qr": ind.c_qr, "c_qi": ind.c_qi}
        return {k: (1.0 - mix) * base[k] + mix * gene[k] for k in base}

    # ------------------------ prediction -------------------
    def _predict_costs_explicit(self, b: int, q: int, N_after: int,
                                c_override: Optional[Dict[str, float]] = None) -> dict:
        n_s = float(self.leaf_size)
        eps = 1e-12

        # geometry snapshot
        if self._n_at_rebuild > 0:
            N_prev = float(self._n_at_rebuild)
            S_prev = float(self._accum_since_rebuild)
        else:
            N_prev = float(max(1, N_after - max(0, b)))
            S_prev = 0.0
        S_after = S_prev + float(b)

        log_after = max(eps, math.log2(max(2.0, float(N_after)) / max(1.0, n_s)))
        n_s_i = n_s * (1.0 + (S_after / max(1.0, float(N_prev))))

        # Insert base pairs (fallback)
        tpi_base  = self.tpi
        tpqI_base = self.tpqI
        if tpi_base is None and tpqI_base is not None:
            tpi_base = float(max(eps, tpqI_base / n_s_i))
        if tpqI_base is None and tpi_base is not None:
            tpqI_base = float(max(eps, tpi_base * n_s_i))

        tpqR_base = self.tpqR
        tpr_prev  = self.tpr

        # safe defaults
        def _safe(x, fb): return float(x) if (x is not None and math.isfinite(x) and x > 0) else float(fb)
        tpi  = _safe(tpi_base,  1e-7)
        tpqI = _safe(tpqI_base, 1e-8)
        tpqR = _safe(tpqR_base, 1e-8)

        #  choose scalers in effect
        if c_override is None:
            c_pr_loc, c_pi_loc, c_qr_loc, c_qi_loc = self._c_pr, self._c_pi, self._c_qr, self._c_qi
        else:
            c_pr_loc = max(self.cpr_floor_min, float(c_override.get("c_pr", self._c_pr)))
            c_pi_loc = float(c_override.get("c_pi", self._c_pi))
            c_qr_loc = float(c_override.get("c_qr", self._c_qr))
            c_qi_loc = float(c_override.get("c_qi", self._c_qi))

        # ---------- tpr prediction ----------
        if (self._n_at_rebuild > 0) and (tpr_prev is not None):
            tpr_pred = float(tpr_prev) * float(c_pr_loc)
        else:
            tpr_pred = 1e-7

        # effective unit times
        tpr_eff  = tpr_pred
        tpqR_eff = tpqR * c_qr_loc
        tpi_eff  = tpi  * c_pi_loc
        tpqI_eff = tpqI * c_qi_loc

        # rebuild totals
        T_rebuild_op = tpr_eff  * float(N_after) * log_after
        T_rebuild_q  = tpqR_eff * float(q) * (n_s + log_after)
        T_rebuild    = T_rebuild_op + T_rebuild_q

        # insert totals
        NR = float(max(1, self._n_at_rebuild if self._n_at_rebuild > 0 else N_after - max(0, b)))
        logNR = max(eps, math.log2(max(2.0, NR) / max(1.0, n_s)))
        T_insert_op = (tpi_eff * float(b) * logNR) if b > 0 else 0.0

        # deterministic velocity for insert queries
        if q > 0 and (self.vq_anchor is not None) and (self.ns_at_anchor is not None) and (self.vq_anchor > 0):
            vq_pred_insert = float(self.vq_anchor) * (float(self.ns_at_anchor) / float(n_s_i))
            if vq_pred_insert > 0:
                T_insert_q = float(q) / vq_pred_insert
            else:
                T_insert_q = tpqI_eff * float(q) * (n_s_i + logNR)
        else:
            vq_pred_insert = None
            T_insert_q = tpqI_eff * float(q) * (n_s_i + logNR)

        T_insert = T_insert_op + T_insert_q

        return {
            "T_rebuild": T_rebuild, "T_insert": T_insert,
            "T_rebuild_op": T_rebuild_op, "T_rebuild_q": T_rebuild_q,
            "T_insert_op": T_insert_op, "T_insert_q": T_insert_q,
            "N_R": int(NR), "S_prev": int(S_prev), "S_after": int(S_after),
            "N_tot_after": int(N_after),
            "logR_tot": log_after, "logR_NR": logNR,
            "n_s_after": float(n_s_i),
            "q_raw": int(q), "q_eff": int(q),
            "tpr_base":  float(tpr_prev) if (tpr_prev is not None) else 1e-7,
            "tpr_eff":   float(tpr_eff),
            "tpqR_base": tpqR,  "tpqR_eff":  tpqR_eff,
            "tpi_base":  tpi,   "tpi_eff":  tpi_eff,
            "tpqI_base": tpqI,  "tpqI_eff": tpqI_eff,
            # expose the scalers used in this prediction:
            "c_pr": c_pr_loc, "c_pi": c_pi_loc, "c_qr": c_qr_loc, "c_qi": c_qi_loc,
            "epoch_id": self._epoch_id,
            "vq_anchor": float(self.vq_anchor) if self.vq_anchor is not None else -1.0,
            "ns_at_anchor": float(self.ns_at_anchor) if self.ns_at_anchor is not None else -1.0,
            "vq_pred_insert": float(vq_pred_insert) if vq_pred_insert is not None else -1.0,
        }

    # ---------------------------- decision --------------
    def should_rebuild(self, batch_size: int, query_size: int, n_total_after: int) -> bool:
        b = max(0, int(batch_size))
        q = max(0, int(query_size))
        N_after = max(1, int(n_total_after))

        # Population-as-policy: soft blend of scalers for prediction only
        ind = self._sample_individual()
        c_used = self._blend_scalers(ind, self.policy_mix)

        pred = self._predict_costs_explicit(b, q, N_after, c_override=c_used)
        self.last_prediction = pred
        self.total_decisions += 1
        self._decisions_since_evolve += 1

        # start-of-run rule
        if self._n_at_rebuild <= 0:
            act_rebuild = (b > 0)
            # remember what was used for EA loss later
            self._last_policy = {"ind": ind, "c_used": c_used, "pred": pred, "act": "rebuild" if act_rebuild else "insert"}
            return act_rebuild

        # no double rebuilds option for possible cold starts
        if self._last_was_rebuild or self._since_rebuild_batches < self._min_gap_batches:
            self._last_policy = {"ind": ind, "c_used": c_used, "pred": pred, "act": "insert"}
            return False

        # margin rule
        act_rebuild = (pred["T_rebuild"] * self._rebuild_margin < pred["T_insert"])
        self._last_policy = {"ind": ind, "c_used": c_used, "pred": pred, "act": "rebuild" if act_rebuild else "insert"}
        return act_rebuild

    # ---------------------------- learning & velocity ---------
    def update(self,
               batch_size: int,
               query_size: int,
               did_rebuild: bool,
               operation_time: float,
               query_time: float,
               n_total_after: int):

        b = max(0, int(batch_size))
        q = max(0, int(query_size))
        N_after = max(1, int(n_total_after))

        n_s = float(self.leaf_size)
        eps = 1e-12

        # theoretical guardrail for c_pi
        first_after_rebuild = (not did_rebuild) and (self._n_at_rebuild > 0) and (self._since_rebuild_batches == 0)

        # For the gradient/multiplicative updates, recompute a base prediction
        # so the updates behave exactly as before.
        pred_base = self._predict_costs_explicit(b, q, N_after, c_override=None)

        if did_rebuild:
            # ===== epoch start =====
            self._epoch_id += 1
            N_new  = float(N_after)
            log_new = max(eps, math.log2(max(2.0, N_new) / max(1.0, n_s)))

            # ----- AbsErr update for c_pr using rebuild op-time only -----
            if self.last_prediction is not None and self._epoch_id > 1:
                pred_op = float(pred_base.get("T_rebuild_op", 0.0))  # <-- base pred (no EA blend)
                err = float(operation_time) - pred_op  # signed
                if abs(err) <= 1.0:
                    delta = err
                else:
                    denom = max(pred_op, float(operation_time), 1e-12)
                    delta = err / denom
                self._c_pr = max(self.cpr_floor_min, self._c_pr + delta)
                if not math.isfinite(self._c_pr):
                    self._c_pr = self.cpr_floor_min

            # ----- set base tpr for the new epoch from measurement -----
            denom = N_new * log_new
            if denom > 0.0 and operation_time > 0.0:
                self.tpr = max(eps, operation_time / denom)
            elif self.tpr is None:
                self.tpr = 1e-7

            if q > 0 and query_time > 0.0:
                denom_q = float(q) * (n_s + log_new)
                if denom_q > 0:
                    self.tpqR = max(eps, query_time / denom_q)

            # reset epoch geometry/state
            self._n_at_rebuild = int(N_after)
            self._accum_since_rebuild = 0
            self._last_was_rebuild = True
            self._since_rebuild_batches = 0
            self._reb_times.append(max(0.0, operation_time))
            self._tpi_status  = "carry"  if (self.tpi  is not None) else "unknown"
            self._tpqI_status = "carry"  if (self.tpqI is not None) else "unknown"

            # ---- deterministic velocity bookkeeping on rebuild ----
            if q > 0 and query_time > 0.0:
                self.vq_anchor = float(q) / float(query_time)
                self.ns_at_anchor = float(n_s)
            else:
                self.vq_anchor = None
                self.ns_at_anchor = None

            # -------- query scaler updates --------
            if q > 0 and (self.tpqR is not None) and query_time > 0.0:
                base_q = float(self.tpqR) * float(q) * (n_s + log_new)
                pred_q = base_q * self._c_qr
                if base_q > 0:
                    r = query_time / max(pred_q, 1e-18)
                    self._hist_logs["reb_q"].append(math.log(max(r, 1e-18)))
                    r_cat = _winsor_logmean(list(self._hist_logs["reb_q"]))
                    self._c_qr = float(self._c_qr) * (r ** (self._alpha_base)) * (r_cat ** (self._alpha_cat))

        else:
            # ===== insert inside epoch =====
            self._accum_since_rebuild += b
            self._last_was_rebuild = False
            self._since_rebuild_batches += 1
            self._ins_times.append(max(0.0, operation_time))

            NR = float(max(1, self._n_at_rebuild))
            logNR = max(eps, math.log2(max(2.0, NR) / max(1.0, n_s)))
            S_after = float(self._accum_since_rebuild)
            n_s_i = n_s * (1.0 + (S_after / max(1.0, NR)))

            # exact tpi if b>0
            if (b > 0) and (operation_time > 0.0) and (self._tpi_status != "exact"):
                denom_op = float(b) * logNR
                if denom_op > 0:
                    self.tpi = max(eps, operation_time / denom_op)
                    self._tpi_status = "exact"

            # exact tpqI if q>0
            if (q > 0) and (query_time > 0.0) and (self._tpqI_status != "exact"):
                denom_q = float(q) * (n_s_i + logNR)
                if denom_q > 0:
                    self.tpqI = max(eps, query_time / denom_q)
                    self._tpqI_status = "exact"

            # algebraic couplings
            if (b > 0) and (self._tpqI_status != "exact") and (self.tpi is not None):
                self.tpqI = max(eps, self.tpi * n_s_i)
                if self._tpqI_status == "unknown":
                    self._tpqI_status = "approx"
            if (q > 0) and (self._tpi_status != "exact") and (self.tpqI is not None):
                self.tpi = max(eps, self.tpqI / n_s_i)
                if self._tpi_status == "unknown":
                    self._tpi_status = "approx"

            # multiplicative scaler updates (insert op)
            if (self.tpi is not None) and (b > 0) and (operation_time > 0.0):
                base_op = float(self.tpi) * float(b) * logNR
                pred_op = base_op * self._c_pi
                if base_op > 0.0:
                    r = operation_time / max(pred_op, 1e-18)
                    self._hist_logs["ins_op"].append(math.log(max(r, 1e-18)))
                    r_cat = _winsor_logmean(list(self._hist_logs["ins_op"]))
                    self._c_pi = float(self._c_pi) * (r ** (self._alpha_base)) * (r_cat ** self._alpha_cat)

                    # ---- tight no-double-rebuild guardrail (only first insert after a rebuild) ----
                    if first_after_rebuild:
                        n  = float(self._n_at_rebuild)
                        ns = float(self.leaf_size)
                        nb = n + float(b)

                        log_n  = max(eps, math.log2(max(2.0, n)  / max(1.0, ns)))
                        log_nb = max(eps, math.log2(max(2.0, nb) / max(1.0, ns)))

                        tpr = float(self.tpr) if (self.tpr is not None and math.isfinite(self.tpr) and self.tpr > 0.0) else None
                        tpi = float(self.tpi) if (self.tpi is not None and math.isfinite(self.tpi) and self.tpi > 0.0) else None

                        if (tpr is not None) and (tpi is not None) and (log_n > 0.0) and (log_nb > 0.0):
                            Gamma_tight = (tpr / tpi) * (nb * log_nb) / (float(b) * log_n)
                            cap = float(self._c_pr) * float(Gamma_tight)
                            if self._c_pi > cap:
                                self._c_pi = cap
            # -----------------------------------------------------------------------
            if (self.tpqI is not None) and (q > 0) and (query_time > 0.0):
                base_q = float(self.tpqI) * float(q) * (n_s_i + logNR)
                pred_q = base_q * self._c_qi
                if base_q > 0:
                    r = query_time / max(pred_q, 1e-18)
                    self._hist_logs["ins_q"].append(math.log(max(r, 1e-18)))
                    r_cat = _winsor_logmean(list(self._hist_logs["ins_q"]))
                    self._c_qi = float(self._c_qi) * (r ** (self._alpha_base)) * (r_cat ** self._alpha_cat)

            # -----------------------------------------------------------------------
            # deterministic velocity updates
            if q > 0 and query_time > 0.0:
                self.vq_anchor = float(q) / float(query_time)
                self.ns_at_anchor = float(n_s_i)
            else:
                if (b > 0) and (self.vq_anchor is not None) and (self.ns_at_anchor is not None):
                    self.vq_anchor = float(self.vq_anchor) * (float(self.ns_at_anchor) / float(n_s_i))
                    self.ns_at_anchor = float(n_s_i)

        # ---------------- EA loss + evolve ----------------
        # Gate EA credit assignment until we have at least one real rebuild-calibrated epoch.
        if (self._last_policy is not None) and (self.tpr is not None) and (self._n_at_rebuild > 0):
            pol = self._last_policy
            ind = pol["ind"]
            pred = pol["pred"]
            meas_total = float(operation_time) + float(max(0.0, query_time))

            pred_taken = float(pred["T_rebuild"] if did_rebuild else pred["T_insert"])
            pred_other = float(pred["T_insert"] if did_rebuild else pred["T_rebuild"])

            ratio = meas_total / max(pred_taken, 1e-18)
            wrong_arm = 1.0 if (meas_total > (1.0 + self.wrong_arm_margin) * pred_other) else 0.0

            loss = self.alpha_loss * ratio + (1.0 - self.alpha_loss) * wrong_arm
            ind.update_fit(loss)

        if self._decisions_since_evolve >= self.evolve_every:
            self._evolve()
            self._decisions_since_evolve = 0

        # ---------------- Lamarckian writeback ----------------
        self._updates_since_lamarck += 1
        if self._updates_since_lamarck >= self.lamarck_every:
            self._lamarckian_writeback()
            self._updates_since_lamarck = 0

    # softly pull elites toward the current live scalers
    def _lamarckian_writeback(self):
        if not self.population:
            return
        pop = sorted(self.population, key=self._w, reverse=True)
        k = max(1, int(self.elite_ratio * len(pop)))
        elites = pop[:k]
        lam = self.lamarck_lambda
        for e in elites:
            e.c_pr = max(self.cpr_floor_min, lam * e.c_pr + (1.0 - lam) * self._c_pr)
            e.c_pi = lam * e.c_pi + (1.0 - lam) * self._c_pi
            e.c_qr = lam * e.c_qr + (1.0 - lam) * self._c_qr
            e.c_qi = lam * e.c_qi + (1.0 - lam) * self._c_qi
            # keep within nominal gene box
            e.c_pr = float(min(self.G["c_pr"][1], max(self.G["c_pr"][0], e.c_pr)))
            e.c_pi = float(min(self.G["c_pi"][1], max(self.G["c_pi"][0], e.c_pi)))
            e.c_qr = float(min(self.G["c_qr"][1], max(self.G["c_qr"][0], e.c_qr)))
            e.c_qi = float(min(self.G["c_qi"][1], max(self.G["c_qi"][0], e.c_qi)))

    # ----------------------------- reporter ------------------------
    def get_unit_times(self) -> Dict[str, float]:
        def _s(name, fb):
            val = getattr(self, name, None)
            return float(val) if (val is not None and math.isfinite(val) and val > 0) else float(fb)
        bases = {
            "tpr":  _s("tpr",  1e-7),
            "tpi":  _s("tpi",  1e-7),
            "tpqR": _s("tpqR", 1e-8),
            "tpqI": _s("tpqI", 1e-8),
        }
        c = {"c_pr": self._c_pr, "c_pi": self._c_pi, "c_qr": self._c_qr, "c_qi": self._c_qi}
        eff = {
            "tpr_eff":  bases["tpr"],
            "tpi_eff":  bases["tpi"]  * c["c_pi"],
            "tpqR_eff": bases["tpqR"] * c["c_qr"],
            "tpqI_eff": bases["tpqI"] * c["c_qi"],
        }
        vel = {
            "vq_anchor": float(self.vq_anchor) if (self.vq_anchor is not None) else -1.0,
            "ns_at_anchor": float(self.ns_at_anchor) if (self.ns_at_anchor is not None) else -1.0,
        }
        return { **bases, **c, **eff, **vel,
                 "epoch_id": self._epoch_id,
                 "tpi_status": self._tpi_status, "tpqI_status": self._tpqI_status }

--------------------------------

**-----------VP EA Run 1------------**

In [33]:
# === Fixed Parameters ===
leaf_size_rebuild = 8600
epsilon_rebuild = 0.02
epsilon_insert = 0.7

# Initialize the evolutionary operation selector (matches the velocity logic)
op_selector = EvolutionaryOperationSelector(
    leaf_size=leaf_size_rebuild,
    rebuild_margin=0.8,        # a little eager to rebuild when close
    c_pr_init=0.30,
    seed=42
)

# ---------------- Bookkeeping ----------------
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []     # (batch_num, total_points, points_added, num_queries, b_time, i_time, s_time_wall, s_time_loop, total_time_wall, op, vq_meas_loop)

grand_total_wall = 0.0
cumulative_data = []
vptree = None  # created on first build
must_prepare = False

def _fmt(x):
    import numpy as _np
    return f"{x:.6f}" if isinstance(x, (int, float, _np.floating)) else str(x)

# ===================== Runner Loop =====================
for data_size, query_load in test_combinations:
    batch_idx += 1
    print(f"\nBatch {batch_idx}: Adding {data_size:,} points with {query_load} queries")

    # ----- slice batch -----
    start_idx = sum(size for size, _ in test_combinations[:batch_idx-1])
    end_idx   = start_idx + data_size
    if data_size > 0:
        new_batch = combined_dataset[start_idx:end_idx]
    else:
        new_batch = np.array([]).reshape(0, combined_dataset.shape[1])
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # total points *after* this batch, before executing op
    current_total_points = (len(cumulative_data) if len(cumulative_data) > 0 else 0) + batch_size

    # ----- prediction snapshot (before executing op) -----
    if vptree is None:
        pred = op_selector._predict_costs_explicit(batch_size, query_load, current_total_points)
        op_selector.last_prediction = pred
        should_rebuild = True
    else:
        should_rebuild = op_selector.should_rebuild(
            batch_size=batch_size,
            query_size=query_load,
            n_total_after=current_total_points
        )
        pred = op_selector.last_prediction  # set by should_rebuild()

    # velocity snapshot
    ut = op_selector.get_unit_times()
    vq_anchor   = ut["vq_anchor"]
    ns_anchor   = ut["ns_at_anchor"]
    vq_pred_ins = pred.get("vq_pred_insert", -1.0)

    print("  ── Predictor snapshot (before operation) ──")
    print(f"    Geometry: N_R={pred['N_R']:,}, S_prev={pred['S_prev']:,}, S_after={pred['S_after']:,}, "
          f"N_tot_after={pred['N_tot_after']:,}, logR_tot={_fmt(pred['logR_tot'])}, "
          f"logR_NR={_fmt(pred['logR_NR'])}, n_s_after={_fmt(pred['n_s_after'])}, "
          f"q={pred['q_raw']} (q_eff={pred['q_eff']})")
    print("    Velocity:")
    print(f"      v_q_anchor={_fmt(vq_anchor)} q/s at n_s_anchor={_fmt(ns_anchor)} ; "
          f"v_q_pred_if_INSERT={_fmt(vq_pred_ins)} q/s")
    print("    Unit times (base → effective):")
    print(f"      tpr:   {_fmt(pred['tpr_base'])}  →  {_fmt(pred['tpr_eff'])}")
    print(f"      tpqR:  {_fmt(pred['tpqR_base'])} →  {_fmt(pred['tpqR_eff'])}")
    print(f"      tpi:   {_fmt(pred['tpi_base'])}  →  {_fmt(pred['tpi_eff'])}")
    print(f"      tpqI:  {_fmt(pred['tpqI_base'])} →  {_fmt(pred['tpqI_eff'])}  (fallback only if v_q unknown)")
    print("    Predicted totals:")
    print(f"      Rebuild: T_op={_fmt(pred['T_rebuild_op'])}, T_q={_fmt(pred['T_rebuild_q'])}, T_total={_fmt(pred['T_rebuild'])}")
    print(f"      Insert:  T_op={_fmt(pred['T_insert_op'])},  T_q={_fmt(pred['T_insert_q'])},  T_total={_fmt(pred['T_insert'])}")
    print(f"    Decision: {'REBUILD' if should_rebuild else 'INSERT'}")

    # ----- execute operation -----
    rebuilt = False
    insert_time = 0.0
    build_time  = 0.0

    if should_rebuild or vptree is None:
        rebuilt = True

        # merge data first (old + new)
        if len(cumulative_data) > 0 and batch_size > 0:
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        elif batch_size > 0:
            cumulative_data = new_batch
        else:
            cumulative_data = cumulative_data if len(cumulative_data) > 0 else np.array([]).reshape(0, combined_dataset.shape[1])

        if len(cumulative_data) > 0:
            must_prepare = True
            build_start = time.perf_counter()
            vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
            vptree.build(cumulative_data)
            build_time = time.perf_counter() - build_start

            points_at_last_rebuild = len(cumulative_data)
            accumulated_since_rebuild = 0
        else:
            build_time = 0.0
            must_prepare = False

        print(f"Rebuilding VP Tree from scratch with {len(cumulative_data):,} points.")
        print(f"Build Time: {build_time:.4f} sec")

    else:
        # switch epsilon for inserts
        if vptree is not None:
            vptree.epsilon = epsilon_insert

        if batch_size > 0:
            must_prepare = True
            insert_start = time.perf_counter()
            for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points (Batch {batch_idx})"):
                vptree.insert(point)
            insert_time = time.perf_counter() - insert_start

            # merge new data into cumulative
            if len(cumulative_data) > 0:
                cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
            else:
                cumulative_data = new_batch

            print(f"Insert Time: {insert_time:.4f} sec")
        else:
            insert_time = 0.0
            print(f"No points to insert - Processing time: {insert_time:.4f} sec")

    # ----- queries -----
    prep_time = 0.0
    loop_time = 0.0
    total_search_time_wall = 0.0
    vq_meas_loop = -1.0

    queries = queries_list[batch_idx-1] if query_load > 0 else []
    if vptree is not None and query_load > 0:
        # prepare-for-search timing (EXCLUDED from v_q learning)
        if must_prepare:
            t0 = time.perf_counter()
            vptree.prepare_for_search()
            prep_time = time.perf_counter() - t0
            must_prepare = False

        # loop-only timing (INCLUDED in v_q learning)
        t_loop_start = time.perf_counter()
        tbar = tqdm(queries, desc=f"Querying batch {batch_idx} ({len(queries)} queries)")
        for query in tbar:
            r_min, nn_pt = vptree.search(query=query)
        loop_time = time.perf_counter() - t_loop_start

        # take tqdm’s rate if available; fallback to loop_time
        rate_from_tqdm = 0.0
        try:
            rate_from_tqdm = float(tbar.format_dict.get("rate", 0.0)) or 0.0
        except Exception:
            rate_from_tqdm = 0.0
        vq_meas_loop = rate_from_tqdm if rate_from_tqdm > 0 else ((len(queries) / loop_time) if loop_time > 0 else -1.0)

        total_search_time_wall = prep_time + loop_time
    else:
        total_search_time_wall = 0.0
        vq_meas_loop = -1.0

    print(f"Total Search Time (wall = prep+loop): {total_search_time_wall:.6f} sec")
    if query_load > 0 and vq_meas_loop > 0:
        print(f"    Measured v_q(loop only)={vq_meas_loop:.2f} q/s")

    # ----- update selector with measured performance -----
    # IMPORTANT: pass loop-only time for query_time so v_q learning uses true loop throughput
    op_selector.update(
        batch_size=batch_size,
        query_size=query_load,
        did_rebuild=rebuilt,
        operation_time=(build_time if rebuilt else insert_time),
        query_time=loop_time,                   # <-- loop-only
        n_total_after=current_total_points
    )

    # ----- prediction vs measured (apples-to-apples) -----
    # Model predicts op + LOOP (not prep). So compare to op + loop only.
    pred_total_model = pred["T_rebuild"] if rebuilt else pred["T_insert"]
    measured_model   = (build_time if rebuilt else insert_time) + loop_time

    abs_err_model = measured_model - pred_total_model
    rel_err_model = (abs_err_model / pred_total_model) if pred_total_model != 0 else float('inf')

    # Also show wall totals for context (op + prep + loop)
    measured_wall = (build_time if rebuilt else insert_time) + total_search_time_wall

    print("  ── Prediction vs Measured (Model Scope) ──")
    print(f"    Predicted (op+loop): {pred_total_model:.6f}  |  Measured (op+loop): {measured_model:.6f}  "
          f"|  AbsErr: {abs_err_model:.6f}  RelErr: {rel_err_model:.2%}")
    if abs(total_search_time_wall - loop_time) > 1e-9:
        print(f"    Note: prep_time={prep_time:.6f} s included in wall total but excluded from model.")

    # ----- accumulate summary -----
    total_batch_time_wall = (build_time + insert_time) + total_search_time_wall
    grand_total_wall += total_batch_time_wall

    results.append((
        batch_idx,
        len(cumulative_data),
        data_size,
        len(queries),
        build_time,
        insert_time,
        total_search_time_wall,
        loop_time,
        total_batch_time_wall,
        "rebuild" if rebuilt else "insert",
        vq_meas_loop
    ))

# ===================== Summary =====================
print("\n=== Summary of VP Tree Scaling Test with Deterministic v_q ===")
for (batch_num, total_points, points_added, num_queries, b_time, i_time, s_time_wall, s_time_loop, total_time_wall, operation, vq_meas) in results:
    vq_str = f", v_q_meas(loop)={vq_meas:,.2f} q/s" if (num_queries > 0 and vq_meas > 0) else ""
    if operation == "rebuild":
        print(f"Batch {batch_num}: Added {points_added:,} points ({total_points:,} total), {num_queries} queries, "
              f"Operation: {operation}, Build Time: {b_time:.4f} sec, "
              f"Search Time(loop)={s_time_loop:.6f} sec, Search Time(wall)={s_time_wall:.6f} sec, "
              f"Total Wall Time: {total_time_wall:.4f} sec{vq_str}")
    else:
        print(f"Batch {batch_num}: Added {points_added:,} points ({total_points:,} total), {num_queries} queries, "
              f"Operation: {operation}, Insert Time: {i_time:.4f} sec, "
              f"Search Time(loop)={s_time_loop:.6f} sec, Search Time(wall)={s_time_wall:.6f} sec, "
              f"Total Wall Time: {total_time_wall:.4f} sec{vq_str}")

print(f"\nGrand Total (wall): {grand_total_wall:.4f} sec")


Batch 1: Adding 10,000 points with 0 queries
  ── Predictor snapshot (before operation) ──
    Geometry: N_R=1, S_prev=0, S_after=10,000, N_tot_after=10,000, logR_tot=0.217591, logR_NR=0.000000, n_s_after=86008600.000000, q=0 (q_eff=0)
    Velocity:
      v_q_anchor=-1.000000 q/s at n_s_anchor=-1.000000 ; v_q_pred_if_INSERT=-1.000000 q/s
    Unit times (base → effective):
      tpr:   0.000000  →  0.000000
      tpqR:  0.000000 →  0.000000
      tpi:   0.000000  →  0.000000
      tpqI:  0.000000 →  0.000000  (fallback only if v_q unknown)
    Predicted totals:
      Rebuild: T_op=0.000218, T_q=0.000000, T_total=0.000218
      Insert:  T_op=0.000000,  T_q=0.000000,  T_total=0.000000
    Decision: REBUILD
Rebuilding VP Tree from scratch with 10,000 points.
Build Time: 0.0167 sec
Total Search Time (wall = prep+loop): 0.000000 sec
  ── Prediction vs Measured (Model Scope) ──
    Predicted (op+loop): 0.000218  |  Measured (op+loop): 0.016736  |  AbsErr: 0.016518  RelErr: 7591.39%

Batch 2:

Inserting 100,000 points (Batch 2): 100%|██████████| 100000/100000 [00:00<00:00, 157291.98it/s]


Insert Time: 0.6389 sec


Querying batch 2 (100 queries): 100%|██████████| 100/100 [00:00<00:00, 129.90it/s]


Total Search Time (wall = prep+loop): 0.808305 sec
    Measured v_q(loop only)=129.40 q/s
  ── Prediction vs Measured (Model Scope) ──
    Predicted (op+loop): 0.095157  |  Measured (op+loop): 1.411757  |  AbsErr: 1.316599  RelErr: 1383.60%
    Note: prep_time=0.035493 s included in wall total but excluded from model.

Batch 3: Adding 0 points with 100 queries
  ── Predictor snapshot (before operation) ──
    Geometry: N_R=10,000, S_prev=100,000, S_after=100,000, N_tot_after=110,000, logR_tot=3.677023, logR_NR=0.217591, n_s_after=94600.000000, q=100 (q_eff=100)
    Velocity:
      v_q_anchor=129.397597 q/s at n_s_anchor=94600.000000 ; v_q_pred_if_INSERT=129.397597 q/s
    Unit times (base → effective):
      tpr:   0.000008  →  0.000003
      tpqR:  0.000000 →  0.000000
      tpi:   0.000029  →  0.000030
      tpqI:  0.000000 →  0.000000  (fallback only if v_q unknown)
    Predicted totals:
      Rebuild: T_op=1.378182, T_q=0.008529, T_total=1.386711
      Insert:  T_op=0.000000,  T_q=

...

Total Search Time (wall = prep+loop): 2.447054 sec
    Measured v_q(loop only)=240.16 q/s
  ── Prediction vs Measured (Model Scope) ──
    Predicted (op+loop): 35.579563  |  Measured (op+loop): 21.543745  |  AbsErr: -14.035819  RelErr: -39.45%
    Note: prep_time=0.365107 s included in wall total but excluded from model.

Batch 9: Adding 100,000 points with 0 queries
  ── Predictor snapshot (before operation) ──
    Geometry: N_R=1,420,000, S_prev=0, S_after=100,000, N_tot_after=1,520,000, logR_tot=7.465519, logR_NR=7.367339, n_s_after=9205.633803, q=0 (q_eff=0)
    Velocity:
      v_q_anchor=240.159812 q/s at n_s_anchor=8600.000000 ; v_q_pred_if_INSERT=-1.000000 q/s
    Unit times (base → effective):
      tpr:   0.000002  →  0.000001
      tpqR:  0.000000 →  0.000000
      tpi:   0.000029  →  0.000030
      tpqI:  0.000000 →  0.000000  (fallback only if v_q unknown)
    Predicted totals:
      Rebuild: T_op=9.235611, T_q=0.000000, T_total=9.235611
      Insert:  T_op=22.293955,  T_q=

Inserting 100,000 points (Batch 9): 100%|██████████| 100000/100000 [00:02<00:00, 34297.34it/s]


Insert Time: 2.9201 sec
Total Search Time (wall = prep+loop): 0.000000 sec
  ── Prediction vs Measured (Model Scope) ──
    Predicted (op+loop): 22.293955  |  Measured (op+loop): 2.920090  |  AbsErr: -19.373865  RelErr: -86.90%

Batch 10: Adding 0 points with 20 queries
  ── Predictor snapshot (before operation) ──
    Geometry: N_R=1,420,000, S_prev=100,000, S_after=100,000, N_tot_after=1,520,000, logR_tot=7.465519, logR_NR=7.367339, n_s_after=9205.633803, q=20 (q_eff=20)
    Velocity:
      v_q_anchor=224.359825 q/s at n_s_anchor=9205.633803 ; v_q_pred_if_INSERT=224.359825 q/s
    Unit times (base → effective):
      tpr:   0.000002  →  0.000001
      tpqR:  0.000000 →  0.000000
      tpi:   0.000004  →  0.000004
      tpqI:  0.036487 →  0.037200  (fallback only if v_q unknown)
    Predicted totals:
      Rebuild: T_op=9.235611, T_q=0.083865, T_total=9.319476
      Insert:  T_op=0.000000,  T_q=0.089143,  T_total=0.089143
    Decision: INSERT
No points to insert - Processing time: 0.0

Querying batch 10 (20 queries): 100%|██████████| 20/20 [00:00<00:00, 250.69it/s]

Total Search Time (wall = prep+loop): 0.533152 sec
    Measured v_q(loop only)=241.63 q/s
  ── Prediction vs Measured (Model Scope) ──
    Predicted (op+loop): 0.089143  |  Measured (op+loop): 0.082773  |  AbsErr: -0.006370  RelErr: -7.15%
    Note: prep_time=0.450380 s included in wall total but excluded from model.

=== Summary of VP Tree Scaling Test with Deterministic v_q ===
Batch 1: Added 10,000 points (10,000 total), 0 queries, Operation: rebuild, Build Time: 0.0167 sec, Search Time(loop)=0.000000 sec, Search Time(wall)=0.000000 sec, Total Wall Time: 0.0167 sec
Batch 2: Added 100,000 points (110,000 total), 100 queries, Operation: insert, Insert Time: 0.6389 sec, Search Time(loop)=0.772812 sec, Search Time(wall)=0.808305 sec, Total Wall Time: 1.4472 sec, v_q_meas(loop)=129.40 q/s
Batch 3: Added 0 points (110,000 total), 100 queries, Operation: insert, Insert Time: 0.0000 sec, Search Time(loop)=0.783766 sec, Search Time(wall)=0.783766 sec, Total Wall Time: 0.7838 sec, v_q_meas(lo

**-----------VP EA Run 2------------**

In [37]:
# === Fixed Parameters ===
leaf_size_rebuild = 8600
epsilon_rebuild = 0.02
epsilon_insert = 0.7

# Initialize the evolutionary operation selector (matches the velocity logic)
op_selector = EvolutionaryOperationSelector(
    leaf_size=leaf_size_rebuild,
    rebuild_margin=0.8,        # a little eager to rebuild when close
    c_pr_init=0.30,
    seed=42
)

# ---------------- Bookkeeping ----------------
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []     # (batch_num, total_points, points_added, num_queries, b_time, i_time, s_time_wall, s_time_loop, total_time_wall, op, vq_meas_loop)

grand_total_wall = 0.0
cumulative_data = []
vptree = None  # created on first build
must_prepare = False

def _fmt(x):
    import numpy as _np
    return f"{x:.6f}" if isinstance(x, (int, float, _np.floating)) else str(x)

# ===================== Runner Loop =====================
for data_size, query_load in test_combinations:
    batch_idx += 1
    print(f"\nBatch {batch_idx}: Adding {data_size:,} points with {query_load} queries")

    # ----- slice batch -----
    start_idx = sum(size for size, _ in test_combinations[:batch_idx-1])
    end_idx   = start_idx + data_size
    if data_size > 0:
        new_batch = combined_dataset[start_idx:end_idx]
    else:
        new_batch = np.array([]).reshape(0, combined_dataset.shape[1])
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # total points *after* this batch, before executing op
    current_total_points = (len(cumulative_data) if len(cumulative_data) > 0 else 0) + batch_size

    # ----- prediction snapshot (before executing op) -----
    if vptree is None:
        pred = op_selector._predict_costs_explicit(batch_size, query_load, current_total_points)
        op_selector.last_prediction = pred
        should_rebuild = True
    else:
        should_rebuild = op_selector.should_rebuild(
            batch_size=batch_size,
            query_size=query_load,
            n_total_after=current_total_points
        )
        pred = op_selector.last_prediction  # set by should_rebuild()

    # velocity snapshot
    ut = op_selector.get_unit_times()
    vq_anchor   = ut["vq_anchor"]
    ns_anchor   = ut["ns_at_anchor"]
    vq_pred_ins = pred.get("vq_pred_insert", -1.0)

    print("  ── Predictor snapshot (before operation) ──")
    print(f"    Geometry: N_R={pred['N_R']:,}, S_prev={pred['S_prev']:,}, S_after={pred['S_after']:,}, "
          f"N_tot_after={pred['N_tot_after']:,}, logR_tot={_fmt(pred['logR_tot'])}, "
          f"logR_NR={_fmt(pred['logR_NR'])}, n_s_after={_fmt(pred['n_s_after'])}, "
          f"q={pred['q_raw']} (q_eff={pred['q_eff']})")
    print("    Velocity:")
    print(f"      v_q_anchor={_fmt(vq_anchor)} q/s at n_s_anchor={_fmt(ns_anchor)} ; "
          f"v_q_pred_if_INSERT={_fmt(vq_pred_ins)} q/s")
    print("    Unit times (base → effective):")
    print(f"      tpr:   {_fmt(pred['tpr_base'])}  →  {_fmt(pred['tpr_eff'])}")
    print(f"      tpqR:  {_fmt(pred['tpqR_base'])} →  {_fmt(pred['tpqR_eff'])}")
    print(f"      tpi:   {_fmt(pred['tpi_base'])}  →  {_fmt(pred['tpi_eff'])}")
    print(f"      tpqI:  {_fmt(pred['tpqI_base'])} →  {_fmt(pred['tpqI_eff'])}  (fallback only if v_q unknown)")
    print("    Predicted totals:")
    print(f"      Rebuild: T_op={_fmt(pred['T_rebuild_op'])}, T_q={_fmt(pred['T_rebuild_q'])}, T_total={_fmt(pred['T_rebuild'])}")
    print(f"      Insert:  T_op={_fmt(pred['T_insert_op'])},  T_q={_fmt(pred['T_insert_q'])},  T_total={_fmt(pred['T_insert'])}")
    print(f"    Decision: {'REBUILD' if should_rebuild else 'INSERT'}")

    # ----- execute operation -----
    rebuilt = False
    insert_time = 0.0
    build_time  = 0.0

    if should_rebuild or vptree is None:
        rebuilt = True

        # merge data first (old + new)
        if len(cumulative_data) > 0 and batch_size > 0:
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        elif batch_size > 0:
            cumulative_data = new_batch
        else:
            cumulative_data = cumulative_data if len(cumulative_data) > 0 else np.array([]).reshape(0, combined_dataset.shape[1])

        if len(cumulative_data) > 0:
            must_prepare = True
            build_start = time.perf_counter()
            vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
            vptree.build(cumulative_data)
            build_time = time.perf_counter() - build_start

            points_at_last_rebuild = len(cumulative_data)
            accumulated_since_rebuild = 0
        else:
            build_time = 0.0
            must_prepare = False

        print(f"Rebuilding VP Tree from scratch with {len(cumulative_data):,} points.")
        print(f"Build Time: {build_time:.4f} sec")

    else:
        # switch epsilon for inserts
        if vptree is not None:
            vptree.epsilon = epsilon_insert

        if batch_size > 0:
            must_prepare = True
            insert_start = time.perf_counter()
            for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points (Batch {batch_idx})"):
                vptree.insert(point)
            insert_time = time.perf_counter() - insert_start

            # merge new data into cumulative
            if len(cumulative_data) > 0:
                cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
            else:
                cumulative_data = new_batch

            print(f"Insert Time: {insert_time:.4f} sec")
        else:
            insert_time = 0.0
            print(f"No points to insert - Processing time: {insert_time:.4f} sec")

    # ----- queries -----
    prep_time = 0.0
    loop_time = 0.0
    total_search_time_wall = 0.0
    vq_meas_loop = -1.0

    queries = queries_list[batch_idx-1] if query_load > 0 else []
    if vptree is not None and query_load > 0:
        # prepare-for-search timing (EXCLUDED from v_q learning)
        if must_prepare:
            t0 = time.perf_counter()
            vptree.prepare_for_search()
            prep_time = time.perf_counter() - t0
            must_prepare = False

        # loop-only timing (INCLUDED in v_q learning)
        t_loop_start = time.perf_counter()
        tbar = tqdm(queries, desc=f"Querying batch {batch_idx} ({len(queries)} queries)")
        for query in tbar:
            r_min, nn_pt = vptree.search(query=query)
        loop_time = time.perf_counter() - t_loop_start

        # take tqdm’s rate if available; fallback to loop_time
        rate_from_tqdm = 0.0
        try:
            rate_from_tqdm = float(tbar.format_dict.get("rate", 0.0)) or 0.0
        except Exception:
            rate_from_tqdm = 0.0
        vq_meas_loop = rate_from_tqdm if rate_from_tqdm > 0 else ((len(queries) / loop_time) if loop_time > 0 else -1.0)

        total_search_time_wall = prep_time + loop_time
    else:
        total_search_time_wall = 0.0
        vq_meas_loop = -1.0

    print(f"Total Search Time (wall = prep+loop): {total_search_time_wall:.6f} sec")
    if query_load > 0 and vq_meas_loop > 0:
        print(f"    Measured v_q(loop only)={vq_meas_loop:.2f} q/s")

    # ----- update selector with measured performance -----
    # IMPORTANT: pass loop-only time for query_time so v_q learning uses true loop throughput
    op_selector.update(
        batch_size=batch_size,
        query_size=query_load,
        did_rebuild=rebuilt,
        operation_time=(build_time if rebuilt else insert_time),
        query_time=loop_time,                   # <-- loop-only
        n_total_after=current_total_points
    )

    # ----- prediction vs measured (apples-to-apples) -----
    # Model predicts op + LOOP (not prep). So compare to op + loop only.
    pred_total_model = pred["T_rebuild"] if rebuilt else pred["T_insert"]
    measured_model   = (build_time if rebuilt else insert_time) + loop_time

    abs_err_model = measured_model - pred_total_model
    rel_err_model = (abs_err_model / pred_total_model) if pred_total_model != 0 else float('inf')

    # Also show wall totals for context (op + prep + loop)
    measured_wall = (build_time if rebuilt else insert_time) + total_search_time_wall

    print("  ── Prediction vs Measured (Model Scope) ──")
    print(f"    Predicted (op+loop): {pred_total_model:.6f}  |  Measured (op+loop): {measured_model:.6f}  "
          f"|  AbsErr: {abs_err_model:.6f}  RelErr: {rel_err_model:.2%}")
    if abs(total_search_time_wall - loop_time) > 1e-9:
        print(f"    Note: prep_time={prep_time:.6f} s included in wall total but excluded from model.")

    # ----- accumulate summary -----
    total_batch_time_wall = (build_time + insert_time) + total_search_time_wall
    grand_total_wall += total_batch_time_wall

    results.append((
        batch_idx,
        len(cumulative_data),
        data_size,
        len(queries),
        build_time,
        insert_time,
        total_search_time_wall,
        loop_time,
        total_batch_time_wall,
        "rebuild" if rebuilt else "insert",
        vq_meas_loop
    ))

# ===================== Summary =====================
print("\n=== Summary of VP Tree Scaling Test with Deterministic v_q ===")
for (batch_num, total_points, points_added, num_queries, b_time, i_time, s_time_wall, s_time_loop, total_time_wall, operation, vq_meas) in results:
    vq_str = f", v_q_meas(loop)={vq_meas:,.2f} q/s" if (num_queries > 0 and vq_meas > 0) else ""
    if operation == "rebuild":
        print(f"Batch {batch_num}: Added {points_added:,} points ({total_points:,} total), {num_queries} queries, "
              f"Operation: {operation}, Build Time: {b_time:.4f} sec, "
              f"Search Time(loop)={s_time_loop:.6f} sec, Search Time(wall)={s_time_wall:.6f} sec, "
              f"Total Wall Time: {total_time_wall:.4f} sec{vq_str}")
    else:
        print(f"Batch {batch_num}: Added {points_added:,} points ({total_points:,} total), {num_queries} queries, "
              f"Operation: {operation}, Insert Time: {i_time:.4f} sec, "
              f"Search Time(loop)={s_time_loop:.6f} sec, Search Time(wall)={s_time_wall:.6f} sec, "
              f"Total Wall Time: {total_time_wall:.4f} sec{vq_str}")

print(f"\nGrand Total (wall): {grand_total_wall:.4f} sec")


Batch 1: Adding 10,000 points with 0 queries
  ── Predictor snapshot (before operation) ──
    Geometry: N_R=1, S_prev=0, S_after=10,000, N_tot_after=10,000, logR_tot=0.217591, logR_NR=0.000000, n_s_after=86008600.000000, q=0 (q_eff=0)
    Velocity:
      v_q_anchor=-1.000000 q/s at n_s_anchor=-1.000000 ; v_q_pred_if_INSERT=-1.000000 q/s
    Unit times (base → effective):
      tpr:   0.000000  →  0.000000
      tpqR:  0.000000 →  0.000000
      tpi:   0.000000  →  0.000000
      tpqI:  0.000000 →  0.000000  (fallback only if v_q unknown)
    Predicted totals:
      Rebuild: T_op=0.000218, T_q=0.000000, T_total=0.000218
      Insert:  T_op=0.000000,  T_q=0.000000,  T_total=0.000000
    Decision: REBUILD
Rebuilding VP Tree from scratch with 10,000 points.
Build Time: 0.0183 sec
Total Search Time (wall = prep+loop): 0.000000 sec
  ── Prediction vs Measured (Model Scope) ──
    Predicted (op+loop): 0.000218  |  Measured (op+loop): 0.018278  |  AbsErr: 0.018060  RelErr: 8300.04%

Batch 2:

Inserting 100,000 points (Batch 2): 100%|██████████| 100000/100000 [00:00<00:00, 140257.98it/s]


Insert Time: 0.7159 sec


Querying batch 2 (100 queries): 100%|██████████| 100/100 [00:00<00:00, 128.97it/s]


Total Search Time (wall = prep+loop): 0.811217 sec
    Measured v_q(loop only)=128.41 q/s
  ── Prediction vs Measured (Model Scope) ──
    Predicted (op+loop): 0.095157  |  Measured (op+loop): 1.494642  |  AbsErr: 1.399485  RelErr: 1470.71%
    Note: prep_time=0.032440 s included in wall total but excluded from model.

Batch 3: Adding 0 points with 100 queries
  ── Predictor snapshot (before operation) ──
    Geometry: N_R=10,000, S_prev=100,000, S_after=100,000, N_tot_after=110,000, logR_tot=3.677023, logR_NR=0.217591, n_s_after=94600.000000, q=100 (q_eff=100)
    Velocity:
      v_q_anchor=128.406574 q/s at n_s_anchor=94600.000000 ; v_q_pred_if_INSERT=128.406574 q/s
    Unit times (base → effective):
      tpr:   0.000008  →  0.000004
      tpqR:  0.000000 →  0.000000
      tpi:   0.000033  →  0.000033
      tpqI:  0.000000 →  0.000000  (fallback only if v_q unknown)
    Predicted totals:
      Rebuild: T_op=1.505161, T_q=0.008529, T_total=1.513690
      Insert:  T_op=0.000000,  T_q=

...

Total Search Time (wall = prep+loop): 2.391136 sec
    Measured v_q(loop only)=242.84 q/s
  ── Prediction vs Measured (Model Scope) ──
    Predicted (op+loop): 38.853719  |  Measured (op+loop): 21.022519  |  AbsErr: -17.831200  RelErr: -45.89%
    Note: prep_time=0.332160 s included in wall total but excluded from model.

Batch 9: Adding 100,000 points with 0 queries
  ── Predictor snapshot (before operation) ──
    Geometry: N_R=1,420,000, S_prev=0, S_after=100,000, N_tot_after=1,520,000, logR_tot=7.465519, logR_NR=7.367339, n_s_after=9205.633803, q=0 (q_eff=0)
    Velocity:
      v_q_anchor=242.839073 q/s at n_s_anchor=8600.000000 ; v_q_pred_if_INSERT=-1.000000 q/s
    Unit times (base → effective):
      tpr:   0.000002  →  0.000001
      tpqR:  0.000000 →  0.000000
      tpi:   0.000033  →  0.000031
      tpqI:  0.000000 →  0.000000  (fallback only if v_q unknown)
    Predicted totals:
      Rebuild: T_op=8.999163, T_q=0.000000, T_total=8.999163
      Insert:  T_op=23.048056,  T_q=

Inserting 100,000 points (Batch 9): 100%|██████████| 100000/100000 [00:02<00:00, 36136.56it/s]


Insert Time: 2.7694 sec
Total Search Time (wall = prep+loop): 0.000000 sec
  ── Prediction vs Measured (Model Scope) ──
    Predicted (op+loop): 23.048056  |  Measured (op+loop): 2.769415  |  AbsErr: -20.278641  RelErr: -87.98%

Batch 10: Adding 0 points with 20 queries
  ── Predictor snapshot (before operation) ──
    Geometry: N_R=1,420,000, S_prev=100,000, S_after=100,000, N_tot_after=1,520,000, logR_tot=7.465519, logR_NR=7.367339, n_s_after=9205.633803, q=20 (q_eff=20)
    Velocity:
      v_q_anchor=226.862818 q/s at n_s_anchor=9205.633803 ; v_q_pred_if_INSERT=226.862818 q/s
    Unit times (base → effective):
      tpr:   0.000002  →  0.000001
      tpqR:  0.000000 →  0.000000
      tpi:   0.000004  →  0.000004
      tpqI:  0.034604 →  0.035590  (fallback only if v_q unknown)
    Predicted totals:
      Rebuild: T_op=8.999163, T_q=0.082940, T_total=9.082103
      Insert:  T_op=0.000000,  T_q=0.088159,  T_total=0.088159
    Decision: INSERT
No points to insert - Processing time: 0.0

Querying batch 10 (20 queries): 100%|██████████| 20/20 [00:00<00:00, 267.75it/s]

Total Search Time (wall = prep+loop): 0.503090 sec
    Measured v_q(loop only)=260.74 q/s
  ── Prediction vs Measured (Model Scope) ──
    Predicted (op+loop): 0.088159  |  Measured (op+loop): 0.076706  |  AbsErr: -0.011453  RelErr: -12.99%
    Note: prep_time=0.426384 s included in wall total but excluded from model.

=== Summary of VP Tree Scaling Test with Deterministic v_q ===
Batch 1: Added 10,000 points (10,000 total), 0 queries, Operation: rebuild, Build Time: 0.0183 sec, Search Time(loop)=0.000000 sec, Search Time(wall)=0.000000 sec, Total Wall Time: 0.0183 sec
Batch 2: Added 100,000 points (110,000 total), 100 queries, Operation: insert, Insert Time: 0.7159 sec, Search Time(loop)=0.778776 sec, Search Time(wall)=0.811217 sec, Total Wall Time: 1.5271 sec, v_q_meas(loop)=128.41 q/s
Batch 3: Added 0 points (110,000 total), 100 queries, Operation: insert, Insert Time: 0.0000 sec, Search Time(loop)=0.816499 sec, Search Time(wall)=0.816499 sec, Total Wall Time: 0.8165 sec, v_q_meas(l

**----------VP EA Run 3---------**

In [30]:
# === Fixed Parameters ===
leaf_size_rebuild = 8600
epsilon_rebuild = 0.02
epsilon_insert = 0.7

# Initialize the evolutionary operation selector (matches the velocity logic)
op_selector = EvolutionaryOperationSelector(
    leaf_size=leaf_size_rebuild,
    rebuild_margin=0.8,        # a little eager to rebuild when close
    c_pr_init=0.30,
    seed=42
)

# ---------------- Bookkeeping ----------------
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []     # (batch_num, total_points, points_added, num_queries, b_time, i_time, s_time_wall, s_time_loop, total_time_wall, op, vq_meas_loop)

grand_total_wall = 0.0
cumulative_data = []
vptree = None  # created on first build
must_prepare = False

def _fmt(x):
    import numpy as _np
    return f"{x:.6f}" if isinstance(x, (int, float, _np.floating)) else str(x)

# ===================== Runner Loop =====================
for data_size, query_load in test_combinations:
    batch_idx += 1
    print(f"\nBatch {batch_idx}: Adding {data_size:,} points with {query_load} queries")

    # ----- slice batch -----
    start_idx = sum(size for size, _ in test_combinations[:batch_idx-1])
    end_idx   = start_idx + data_size
    if data_size > 0:
        new_batch = combined_dataset[start_idx:end_idx]
    else:
        new_batch = np.array([]).reshape(0, combined_dataset.shape[1])
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # total points *after* this batch, before executing op
    current_total_points = (len(cumulative_data) if len(cumulative_data) > 0 else 0) + batch_size

    # ----- prediction snapshot (before executing op) -----
    if vptree is None:
        pred = op_selector._predict_costs_explicit(batch_size, query_load, current_total_points)
        op_selector.last_prediction = pred
        should_rebuild = True
    else:
        should_rebuild = op_selector.should_rebuild(
            batch_size=batch_size,
            query_size=query_load,
            n_total_after=current_total_points
        )
        pred = op_selector.last_prediction  # set by should_rebuild()

    # velocity snapshot
    ut = op_selector.get_unit_times()
    vq_anchor   = ut["vq_anchor"]
    ns_anchor   = ut["ns_at_anchor"]
    vq_pred_ins = pred.get("vq_pred_insert", -1.0)

    print("  ── Predictor snapshot (before operation) ──")
    print(f"    Geometry: N_R={pred['N_R']:,}, S_prev={pred['S_prev']:,}, S_after={pred['S_after']:,}, "
          f"N_tot_after={pred['N_tot_after']:,}, logR_tot={_fmt(pred['logR_tot'])}, "
          f"logR_NR={_fmt(pred['logR_NR'])}, n_s_after={_fmt(pred['n_s_after'])}, "
          f"q={pred['q_raw']} (q_eff={pred['q_eff']})")
    print("    Velocity:")
    print(f"      v_q_anchor={_fmt(vq_anchor)} q/s at n_s_anchor={_fmt(ns_anchor)} ; "
          f"v_q_pred_if_INSERT={_fmt(vq_pred_ins)} q/s")
    print("    Unit times (base → effective):")
    print(f"      tpr:   {_fmt(pred['tpr_base'])}  →  {_fmt(pred['tpr_eff'])}")
    print(f"      tpqR:  {_fmt(pred['tpqR_base'])} →  {_fmt(pred['tpqR_eff'])}")
    print(f"      tpi:   {_fmt(pred['tpi_base'])}  →  {_fmt(pred['tpi_eff'])}")
    print(f"      tpqI:  {_fmt(pred['tpqI_base'])} →  {_fmt(pred['tpqI_eff'])}  (fallback only if v_q unknown)")
    print("    Predicted totals:")
    print(f"      Rebuild: T_op={_fmt(pred['T_rebuild_op'])}, T_q={_fmt(pred['T_rebuild_q'])}, T_total={_fmt(pred['T_rebuild'])}")
    print(f"      Insert:  T_op={_fmt(pred['T_insert_op'])},  T_q={_fmt(pred['T_insert_q'])},  T_total={_fmt(pred['T_insert'])}")
    print(f"    Decision: {'REBUILD' if should_rebuild else 'INSERT'}")

    # ----- execute operation -----
    rebuilt = False
    insert_time = 0.0
    build_time  = 0.0

    if should_rebuild or vptree is None:
        rebuilt = True

        # merge data first (old + new)
        if len(cumulative_data) > 0 and batch_size > 0:
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        elif batch_size > 0:
            cumulative_data = new_batch
        else:
            cumulative_data = cumulative_data if len(cumulative_data) > 0 else np.array([]).reshape(0, combined_dataset.shape[1])

        if len(cumulative_data) > 0:
            must_prepare = True
            build_start = time.perf_counter()
            vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
            vptree.build(cumulative_data)
            build_time = time.perf_counter() - build_start

            points_at_last_rebuild = len(cumulative_data)
            accumulated_since_rebuild = 0
        else:
            build_time = 0.0
            must_prepare = False

        print(f"Rebuilding VP Tree from scratch with {len(cumulative_data):,} points.")
        print(f"Build Time: {build_time:.4f} sec")

    else:
        # switch epsilon for inserts
        if vptree is not None:
            vptree.epsilon = epsilon_insert

        if batch_size > 0:
            must_prepare = True
            insert_start = time.perf_counter()
            for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points (Batch {batch_idx})"):
                vptree.insert(point)
            insert_time = time.perf_counter() - insert_start

            # merge new data into cumulative
            if len(cumulative_data) > 0:
                cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
            else:
                cumulative_data = new_batch

            print(f"Insert Time: {insert_time:.4f} sec")
        else:
            insert_time = 0.0
            print(f"No points to insert - Processing time: {insert_time:.4f} sec")

    # ----- queries -----
    prep_time = 0.0
    loop_time = 0.0
    total_search_time_wall = 0.0
    vq_meas_loop = -1.0

    queries = queries_list[batch_idx-1] if query_load > 0 else []
    if vptree is not None and query_load > 0:
        # prepare-for-search timing (EXCLUDED from v_q learning)
        if must_prepare:
            t0 = time.perf_counter()
            vptree.prepare_for_search()
            prep_time = time.perf_counter() - t0
            must_prepare = False

        # loop-only timing (INCLUDED in v_q learning)
        t_loop_start = time.perf_counter()
        tbar = tqdm(queries, desc=f"Querying batch {batch_idx} ({len(queries)} queries)")
        for query in tbar:
            r_min, nn_pt = vptree.search(query=query)
        loop_time = time.perf_counter() - t_loop_start

        # take tqdm’s rate if available; fallback to loop_time
        rate_from_tqdm = 0.0
        try:
            rate_from_tqdm = float(tbar.format_dict.get("rate", 0.0)) or 0.0
        except Exception:
            rate_from_tqdm = 0.0
        vq_meas_loop = rate_from_tqdm if rate_from_tqdm > 0 else ((len(queries) / loop_time) if loop_time > 0 else -1.0)

        total_search_time_wall = prep_time + loop_time
    else:
        total_search_time_wall = 0.0
        vq_meas_loop = -1.0

    print(f"Total Search Time (wall = prep+loop): {total_search_time_wall:.6f} sec")
    if query_load > 0 and vq_meas_loop > 0:
        print(f"    Measured v_q(loop only)={vq_meas_loop:.2f} q/s")

    # ----- update selector with measured performance -----
    # IMPORTANT: pass loop-only time for query_time so v_q learning uses true loop throughput
    op_selector.update(
        batch_size=batch_size,
        query_size=query_load,
        did_rebuild=rebuilt,
        operation_time=(build_time if rebuilt else insert_time),
        query_time=loop_time,                   # <-- loop-only
        n_total_after=current_total_points
    )

    # ----- prediction vs measured (apples-to-apples) -----
    # Model predicts op + LOOP (not prep). So compare to op + loop only.
    pred_total_model = pred["T_rebuild"] if rebuilt else pred["T_insert"]
    measured_model   = (build_time if rebuilt else insert_time) + loop_time

    abs_err_model = measured_model - pred_total_model
    rel_err_model = (abs_err_model / pred_total_model) if pred_total_model != 0 else float('inf')

    # Also show wall totals for context (op + prep + loop)
    measured_wall = (build_time if rebuilt else insert_time) + total_search_time_wall

    print("  ── Prediction vs Measured (Model Scope) ──")
    print(f"    Predicted (op+loop): {pred_total_model:.6f}  |  Measured (op+loop): {measured_model:.6f}  "
          f"|  AbsErr: {abs_err_model:.6f}  RelErr: {rel_err_model:.2%}")
    if abs(total_search_time_wall - loop_time) > 1e-9:
        print(f"    Note: prep_time={prep_time:.6f} s included in wall total but excluded from model.")

    # ----- accumulate summary -----
    total_batch_time_wall = (build_time + insert_time) + total_search_time_wall
    grand_total_wall += total_batch_time_wall

    results.append((
        batch_idx,
        len(cumulative_data),
        data_size,
        len(queries),
        build_time,
        insert_time,
        total_search_time_wall,
        loop_time,
        total_batch_time_wall,
        "rebuild" if rebuilt else "insert",
        vq_meas_loop
    ))

# ===================== Summary =====================
print("\n=== Summary of VP Tree Scaling Test with Deterministic v_q ===")
for (batch_num, total_points, points_added, num_queries, b_time, i_time, s_time_wall, s_time_loop, total_time_wall, operation, vq_meas) in results:
    vq_str = f", v_q_meas(loop)={vq_meas:,.2f} q/s" if (num_queries > 0 and vq_meas > 0) else ""
    if operation == "rebuild":
        print(f"Batch {batch_num}: Added {points_added:,} points ({total_points:,} total), {num_queries} queries, "
              f"Operation: {operation}, Build Time: {b_time:.4f} sec, "
              f"Search Time(loop)={s_time_loop:.6f} sec, Search Time(wall)={s_time_wall:.6f} sec, "
              f"Total Wall Time: {total_time_wall:.4f} sec{vq_str}")
    else:
        print(f"Batch {batch_num}: Added {points_added:,} points ({total_points:,} total), {num_queries} queries, "
              f"Operation: {operation}, Insert Time: {i_time:.4f} sec, "
              f"Search Time(loop)={s_time_loop:.6f} sec, Search Time(wall)={s_time_wall:.6f} sec, "
              f"Total Wall Time: {total_time_wall:.4f} sec{vq_str}")

print(f"\nGrand Total (wall): {grand_total_wall:.4f} sec")


Batch 1: Adding 10,000 points with 0 queries
  ── Predictor snapshot (before operation) ──
    Geometry: N_R=1, S_prev=0, S_after=10,000, N_tot_after=10,000, logR_tot=0.217591, logR_NR=0.000000, n_s_after=86008600.000000, q=0 (q_eff=0)
    Velocity:
      v_q_anchor=-1.000000 q/s at n_s_anchor=-1.000000 ; v_q_pred_if_INSERT=-1.000000 q/s
    Unit times (base → effective):
      tpr:   0.000000  →  0.000000
      tpqR:  0.000000 →  0.000000
      tpi:   0.000000  →  0.000000
      tpqI:  0.000000 →  0.000000  (fallback only if v_q unknown)
    Predicted totals:
      Rebuild: T_op=0.000218, T_q=0.000000, T_total=0.000218
      Insert:  T_op=0.000000,  T_q=0.000000,  T_total=0.000000
    Decision: REBUILD
Rebuilding VP Tree from scratch with 10,000 points.
Build Time: 0.0134 sec
Total Search Time (wall = prep+loop): 0.000000 sec
  ── Prediction vs Measured (Model Scope) ──
    Predicted (op+loop): 0.000218  |  Measured (op+loop): 0.013442  |  AbsErr: 0.013224  RelErr: 6077.52%

Batch 2:

Inserting 100,000 points (Batch 2): 100%|██████████| 100000/100000 [00:00<00:00, 155694.15it/s]


Insert Time: 0.6447 sec


Querying batch 2 (100 queries): 100%|██████████| 100/100 [00:00<00:00, 132.46it/s]


Total Search Time (wall = prep+loop): 0.788533 sec
    Measured v_q(loop only)=131.91 q/s
  ── Prediction vs Measured (Model Scope) ──
    Predicted (op+loop): 0.095157  |  Measured (op+loop): 1.402794  |  AbsErr: 1.307636  RelErr: 1374.18%
    Note: prep_time=0.030412 s included in wall total but excluded from model.

Batch 3: Adding 0 points with 100 queries
  ── Predictor snapshot (before operation) ──
    Geometry: N_R=10,000, S_prev=100,000, S_after=100,000, N_tot_after=110,000, logR_tot=3.677023, logR_NR=0.217591, n_s_after=94600.000000, q=100 (q_eff=100)
    Velocity:
      v_q_anchor=131.905067 q/s at n_s_anchor=94600.000000 ; v_q_pred_if_INSERT=131.905067 q/s
    Unit times (base → effective):
      tpr:   0.000006  →  0.000003
      tpqR:  0.000000 →  0.000000
      tpi:   0.000030  →  0.000030
      tpqI:  0.000000 →  0.000000  (fallback only if v_q unknown)
    Predicted totals:
      Rebuild: T_op=1.106919, T_q=0.008529, T_total=1.115448
      Insert:  T_op=0.000000,  T_q=

...

Total Search Time (wall = prep+loop): 2.257359 sec
    Measured v_q(loop only)=260.66 q/s
  ── Prediction vs Measured (Model Scope) ──
    Predicted (op+loop): 28.585006  |  Measured (op+loop): 20.351713  |  AbsErr: -8.233293  RelErr: -28.80%
    Note: prep_time=0.339174 s included in wall total but excluded from model.

Batch 9: Adding 100,000 points with 0 queries
  ── Predictor snapshot (before operation) ──
    Geometry: N_R=1,420,000, S_prev=0, S_after=100,000, N_tot_after=1,520,000, logR_tot=7.465519, logR_NR=7.367339, n_s_after=9205.633803, q=0 (q_eff=0)
    Velocity:
      v_q_anchor=260.663107 q/s at n_s_anchor=8600.000000 ; v_q_pred_if_INSERT=-1.000000 q/s
    Unit times (base → effective):
      tpr:   0.000002  →  0.000001
      tpqR:  0.000000 →  0.000000
      tpi:   0.000030  →  0.000030
      tpqI:  0.000000 →  0.000000  (fallback only if v_q unknown)
    Predicted totals:
      Rebuild: T_op=8.747645, T_q=0.000000, T_total=8.747645
      Insert:  T_op=21.960737,  T_q=0

Inserting 100,000 points (Batch 9): 100%|██████████| 100000/100000 [00:02<00:00, 36292.26it/s]


Insert Time: 2.7578 sec
Total Search Time (wall = prep+loop): 0.000000 sec
  ── Prediction vs Measured (Model Scope) ──
    Predicted (op+loop): 21.960737  |  Measured (op+loop): 2.757776  |  AbsErr: -19.202960  RelErr: -87.44%

Batch 10: Adding 0 points with 20 queries
  ── Predictor snapshot (before operation) ──
    Geometry: N_R=1,420,000, S_prev=100,000, S_after=100,000, N_tot_after=1,520,000, logR_tot=7.465519, logR_NR=7.367339, n_s_after=9205.633803, q=20 (q_eff=20)
    Velocity:
      v_q_anchor=243.514219 q/s at n_s_anchor=9205.633803 ; v_q_pred_if_INSERT=243.514219 q/s
    Unit times (base → effective):
      tpr:   0.000002  →  0.000001
      tpqR:  0.000000 →  0.000000
      tpi:   0.000004  →  0.000004
      tpqI:  0.034459 →  0.035681  (fallback only if v_q unknown)
    Predicted totals:
      Rebuild: T_op=8.747645, T_q=0.077268, T_total=8.824913
      Insert:  T_op=0.000000,  T_q=0.082131,  T_total=0.082131
    Decision: INSERT
No points to insert - Processing time: 0.0

Querying batch 10 (20 queries): 100%|██████████| 20/20 [00:00<00:00, 274.28it/s]

Total Search Time (wall = prep+loop): 0.481294 sec
    Measured v_q(loop only)=267.08 q/s
  ── Prediction vs Measured (Model Scope) ──
    Predicted (op+loop): 0.082131  |  Measured (op+loop): 0.074884  |  AbsErr: -0.007247  RelErr: -8.82%
    Note: prep_time=0.406410 s included in wall total but excluded from model.

=== Summary of VP Tree Scaling Test with Deterministic v_q ===
Batch 1: Added 10,000 points (10,000 total), 0 queries, Operation: rebuild, Build Time: 0.0134 sec, Search Time(loop)=0.000000 sec, Search Time(wall)=0.000000 sec, Total Wall Time: 0.0134 sec
Batch 2: Added 100,000 points (110,000 total), 100 queries, Operation: insert, Insert Time: 0.6447 sec, Search Time(loop)=0.758121 sec, Search Time(wall)=0.788533 sec, Total Wall Time: 1.4332 sec, v_q_meas(loop)=131.91 q/s
Batch 3: Added 0 points (110,000 total), 100 queries, Operation: insert, Insert Time: 0.0000 sec, Search Time(loop)=0.760885 sec, Search Time(wall)=0.760885 sec, Total Wall Time: 0.7609 sec, v_q_meas(lo

**-----------VP EA Analysis----------**

In [41]:
import re
import numpy as np

# The result strings
results = [
    'Grand Total (wall): 41.9147',
    'Grand Total (wall): 41.5149',
    'Grand Total (wall): 40.6988'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Grand Total \(wall\):\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)

Total Time - Mean: 41.3761 sec, Std Dev: 0.6197 sec, CV: 1.50%


----------------------------------------

-----------------------------------

### Speedups
---

In [40]:
import re
import math
from typing import Dict, Tuple

def parse_time_stats(s: str) -> Tuple[float, float]:
    """
    Parse strings like:
    'Total Time - Mean: 25.1232 sec, Std Dev: 0.1806'
    Returns (mean, std_dev)
    """
    mean_match = re.search(r"Mean:\s*([0-9.]+)", s)
    std_match  = re.search(r"Std Dev:\s*([0-9.]+)", s)

    if mean_match is None or std_match is None:
        raise ValueError(f"Could not parse stats from: {s}")

    return float(mean_match.group(1)), float(std_match.group(1))


def compute_speedups(
    stats: Dict[str, str],
    runs: Dict[str, int],
    baseline: str = "Exhaustive",
):
    """
    stats: dict of method -> stats string
    runs:  dict of method -> number of runs
    baseline: method name used as denominator

    Prints speedup S_i = T_baseline / T_i and SE(S_i).
    """
    if baseline not in stats:
        raise ValueError("Baseline method not found in stats.")

    # Parse baseline
    T_exh, sd_exh = parse_time_stats(stats[baseline])
    n_exh = runs[baseline]
    se_exh = sd_exh / math.sqrt(n_exh)

    print(f"Baseline: {baseline}")
    print("-" * 60)
    print(f"{'Method':<15} {'S_i':>10} {'SE(S_i)':>12}")
    print("-" * 60)

    for method, s in stats.items():
        if method == baseline:
            print(f"{method:<15} {1.0:>10.3f} {'-':>12}")
            continue

        T_i, sd_i = parse_time_stats(s)
        n_i = runs[method]
        se_i = sd_i / math.sqrt(n_i)

        # Speedup
        S = T_exh / T_i

        # SE via delta method
        se_S = S * math.sqrt(
            (se_exh / T_exh) ** 2 +
            (se_i   / T_i)   ** 2
        )

        print(f"{method:<15} {S:>10.3f} {se_S:>12.3f}")

---

In [42]:
stats = {
    "VP_MWV":     "Total Time - Mean: 613.4250 sec, Std Dev: 24.9538",
    "Exhaustive": "Total Time - Mean: 1907.4550 sec, Std Dev: 7.9550",
    "Log":        "Total Time - Mean: 47.3884 sec, Std Dev: 1.1399",
    "Lamarckian": "Total Time - Mean: 41.3761 sec, Std Dev: 0.6197",
}

runs = {
    "VP_MWV": 2,
    "Exhaustive": 2,
    "Log": 3,
    "Lamarckian": 3,
}

compute_speedups(stats, runs)

Baseline: Exhaustive
------------------------------------------------------------
Method                 S_i      SE(S_i)
------------------------------------------------------------
VP_MWV               3.110        0.090
Exhaustive           1.000            -
Log                 40.252        0.571
Lamarckian          46.100        0.421
